<a href="https://colab.research.google.com/github/zhiruo159/ISOM3260_DatabaseProject/blob/main/%5BISOM3260%5D_Member_Manager_Finalcode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# -- Install and setup Oracle Instant Client --
! wget -O oracleinstantclient.zip https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
! unzip -oq oracleinstantclient.zip
! mkdir -p /opt/oracle
! mv instantclient_19_6 /opt/oracle/
! sh -c "echo /opt/oracle/instantclient_19_6 > /etc/ld.so.conf.d/oracle-instantclient.conf"
! ldconfig


# -- Include oracledb package --
! pip install oracledb

--2025-11-16 06:28:32--  https://download.oracle.com/otn_software/linux/instantclient/19600/instantclient-basic-linux.x64-19.6.0.0.0dbru.zip
Resolving download.oracle.com (download.oracle.com)... 173.222.168.141
Connecting to download.oracle.com (download.oracle.com)|173.222.168.141|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75462547 (72M) [application/zip]
Saving to: ‘oracleinstantclient.zip’

oracleinstantclient 100%[===================>]  71.97M   167MB/s    in 0.4s    

2025-11-16 06:28:33 (167 MB/s) - ‘oracleinstantclient.zip’ saved [75462547/75462547]

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig

In [2]:
# Import oracledb package and connect to Oracle Database
import oracledb

HOST_NAME = "imz409.ust.hk"
PORT_NUMBER = "1521"
SERVICE_NAME = "imz409"
USERNAME = "kcloaf"
PASSWORD = "7745"# Import oracledb package and connect to Oracle Database

dsn_tns = oracledb.makedsn(HOST_NAME, PORT_NUMBER, service_name=SERVICE_NAME)
conn = oracledb.connect(user=USERNAME, password=PASSWORD, dsn=dsn_tns)

c = conn.cursor()

In [9]:
from google.colab.widgets import TabBar
import ipywidgets as widgets
import time
import pandas as pd
from ipywidgets import interact
from ipywidgets import Text
from ipywidgets import DatePicker
from ipywidgets import IntText
from ipywidgets import Label
from datetime import date, datetime

# for Manager Dashboard
#from ipywidgets import Output
from IPython.display import display
from sqlalchemy import create_engine
#import time


# Set current_user_id = None to tell every part of the program "no one is logged in yet"
# It prevents **TyperError: can only concatenate str (not "NoneType") to str** that might happen later on at load_kid_information() tied to build the SQL string while current_user_id was still undefined (or None)
current_user_id = None


# Issac: "I haven't add the tabs for manager at this moment yet"
tb_all = TabBar(['Home Page', 'Manager Login Page', 'Member Login Page', 'Member Register Page',
                 'Main Menu', 'Personal Info', 'Forget Password Page',
                 'Reset Password Page', 'My Kid Information', 'Manager Dashboard', 'Add Teacher',
                 'Edit Teacher', 'Delete Teacher', 'Add Course', 'Edit Course', 'Delete Course','Course List Page','Course Detail Page', 'Enrollment Success Page','Bookmark List Page','Bookmark Added Page','Bookmark Deleted Page'])

# a simple function for jumping between tabs
def showPage(tabName):
    with tb_all.output_to(tabName):
        display()


#######################################################
################ Home Page Tab - Begin ################
#######################################################
with tb_all.output_to('Home Page'):

    # create 3 labels
    title_label = Label("Happy Creche Center Management System")
    subtitle_label = Label("Welcome to our Happy Creche Center Management portal")
    role_label = Label("Please select your role")

    # create 2 button
    btnManagerLoginPage = widgets.Button(description="Continue as Manager")
    btnMemberLoginPage = widgets.Button(description="Continue as Parent")

    # define the event handlers for the two buttons when they are being clicked
    def on_btnManagerLoginPage_button_clicked(b):
        showPage("Manager Login Page")
    btnManagerLoginPage.on_click(on_btnManagerLoginPage_button_clicked)

    def on_btnMemberLoginPage_button_clicked(b):
        showPage("Member Login Page")
    btnMemberLoginPage.on_click(on_btnMemberLoginPage_button_clicked)

    # display the objects
    homePage_row1HBox = widgets.HBox([title_label])
    homePage_row2HBox = widgets.HBox([subtitle_label])
    homePage_row3HBox = widgets.HBox([role_label])
    homePage_row4HBox = widgets.HBox([btnManagerLoginPage])
    homePage_row5HBox = widgets.HBox([btnMemberLoginPage])

    display(widgets.VBox([homePage_row1HBox, homePage_row2HBox, homePage_row3HBox, homePage_row4HBox, homePage_row5HBox]))
################ Home Page Tab - End ################

#######################################################
################ Main Menu Tab - Begin ################
#######################################################
with tb_all.output_to('Main Menu'):

    # create 1 label
    mainMenu_label = Label("Main Menu")
    spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    # create 4 button
    btnKid_InfoPage = widgets.Button(description="Edit my kid Information")
    btnMyProfilePage = widgets.Button(description="Edit my Profile")
    btnCoursePage = widgets.Button(description="View Course")
    btnloginPage = widgets.Button(description="Logout")

    def on_btnKid_InfoPage_button_clicked(b):
        showPage("My Kid Information")
        load_kid_information()  # immediately load kid data
    btnKid_InfoPage.on_click(on_btnKid_InfoPage_button_clicked)

    def on_btnMyProfilePage_button_clicked(b):
        showPage("Personal Info")
        load_member_profile_information()  # immediately load member data
    btnMyProfilePage.on_click(on_btnMyProfilePage_button_clicked)

    def on_btnloginPage_button_clicked(b):
      showPage("Member Login Page")
    btnloginPage.on_click(on_btnloginPage_button_clicked)

    def on_btnCoursePage_button_clicked(b):
      showPage("Course List Page")
    btnCoursePage.on_click(on_btnCoursePage_button_clicked)

    # display the objects
    mainMenu1VBox = widgets.VBox([mainMenu_label])
    mainMenu1HBox = widgets.HBox([btnKid_InfoPage, btnMyProfilePage, btnCoursePage])
    mainMenu2VBox = widgets.VBox([spacer])
    mainMenu2HBox = widgets.HBox([btnloginPage])

    display(widgets.VBox([mainMenu1VBox, mainMenu1HBox, mainMenu2VBox, mainMenu2HBox]))
################ Main Menu Tab - End ################



###########################################################
########### Member Register Page Tab - Begin ##############
###########################################################
with tb_all.output_to('Member Register Page'):

  # create labels and input widgets
  INS_FirstName_label = Label("First Name: ")
  INS_FirstName_textbox = Text(placeholder='First Name', disabled=False)  # False --> textbox is enable, users can type into it

  INS_LastName_label = Label("Last Name: ")
  INS_LastName_textbox = Text(placeholder='Last Name', disabled=False)

  INS_HomeAddress_label = Label("Home Address: ")
  INS_HomeAddress_textbox = Text(placeholder='Home Address', disabled=False)

  INS_DOB_label = Label("DOB: ")
  INS_DOB_datepicker = DatePicker(placeholder='DOB', disabled=False)

  INS_telephone_label = Label("Telephone: ")
  INS_telephone_textbox = Text(placeholder='Telephone', disabled=False)

  INS_email_label = Label("Email: ")
  INS_email_textbox = Text(placeholder='Email', disabled=False)

  INS_password_label = Label("Password: ")
  INS_password_textbox = Text(placeholder='Password', disabled=False)

  Reg_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  INS_Kid_F_Name_label = Label("Kid First Name: ")
  INS_Kid_F_Name_textbox = Text(placeholder='Kid First Name', disabled=False)

  INS_Kid_L_Name_label = Label("Kid Last Name: ")
  INS_Kid_L_Name_textbox = Text(placeholder='Kid Last Name', disabled=False)

  INS_Kid_DOB_label = Label("Kid DOB: ")
  INS_Kid_DOB_datepicker = DatePicker(placeholder='Kid DOB', disabled=False)


  INS_secQ_label = Label("Security Questions")
  INS_secQ_list = widgets.Dropdown(
      options=[
          "What is your favorite fruit?",
          "What is the name of your first pet?",
          "Which country did you travel for the first time?"
      ]
  )

  INS_secAns_label = Label("Security Question Answer")
  INS_secAns_textbox = Text(placeholder='Security Question Answer', disabled=False)

  # create 3 buttons
  INS_back_button = widgets.Button(description="Back")
  INS_clear_button = widgets.Button(description="Clear")
  INS_register_button = widgets.Button(description="Register")

  # event handlers
  def on_INS_back_button_clicked(b):
      showPage("Home Page")
  INS_back_button.on_click(on_INS_back_button_clicked)

  def on_INS_clear_button_clicked(b):
      INS_FirstName_textbox.value = ""
      INS_LastName_textbox.value = ""
      INS_HomeAddress_textbox.value = ""
      INS_DOB_datepicker.value = None
      INS_telephone_textbox.value = ""
      INS_email_textbox.value = ""
      INS_password_textbox.value = ""
      INS_Kid_F_Name_textbox.value = ""
      INS_Kid_L_Name_textbox.value = ""
      INS_Kid_DOB_datepicker.value = ""
      INS_secAns_textbox.value = ""
      INS_secQ_list.value = INS_secQ_list.options[0]
      INS_register_button.disabled = False
  INS_clear_button.on_click(on_INS_clear_button_clicked)

  def on_INS_register_button_clicked(b):
      # generate next Mxxx ID
      c.execute("SELECT MEM_ID FROM MEMBER ORDER BY MEM_ID DESC") # Get all member ID, sort from highest to lowest "DESC" --> M002, M001...
      row = c.fetchone()  # Return only the first row from the query (tuple) result (i.e. all IDs are arranged vertically from the list)
      if row is None:
          MEM_ID = "M001"
      else:
          last_id = row[0]            #M005
          num = int(last_id[1:]) + 1  # --> int('005') + 1 --> 5+1 --> =6
          MEM_ID = f"M{num:03d}"      # 0 --> pad with zeros instead of empty spaces
                                      # 3 --> minimum width of 3 characters
                                      # d --> decimal integer (whole number)

    # insert into database - 14 values, including SYSDATE for MEM_REG_DATE
      c.execute(
        """INSERT INTO MEMBER VALUES ('{}', '{}', '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'),
                                     '{}', '{}', TO_DATE('{}', 'YYYY-MM-DD'),
                                     '{}', '{}', '{}', '{}', '{}', SYSDATE)"""
        .format(
            MEM_ID,                                           # 1. MEM_ID
            INS_password_textbox.value.strip(),               # 2. MEM_PW
            INS_FirstName_textbox.value.strip(),              # 3. MEM_FNAME
            INS_LastName_textbox.value.strip(),               # 4. MEM_LNAME

            # strftime --> str f time --> string from time = "Create a string from time/date object"
            # If a date is selected, convert it to 'YYYY-MM-DD' string; If not, use empty string (Null in database)
            INS_DOB_datepicker.value.strftime('%Y-%m-%d') if INS_DOB_datepicker.value else '',  # 5. MEM_DOB
            INS_Kid_F_Name_textbox.value.strip(),             # 6. K_FNAME
            INS_Kid_L_Name_textbox.value.strip(),             # 7. K_LNAME
            INS_Kid_DOB_datepicker.value.strftime('%Y-%m-%d') if INS_Kid_DOB_datepicker.value else '',  # 8. K_DOB
            INS_secQ_list.value,                              # 9. MEM_SECURITY_Q
            INS_secAns_textbox.value.strip(),                 # 10. MEM_SECURITY_ANS
            INS_telephone_textbox.value.strip(),              # 11. MEM_TEL
            INS_HomeAddress_textbox.value.strip(),            # 12. MEM_ADD
            INS_email_textbox.value.strip()                   # 13. MEM_EMAIL
            # 14. MEM_REG_DATE → SYSDATE (auto-filled)
         )
      )
      c.execute("commit")
      INS_register_button.disabled = True

      # hide form
      INS_line1HBox.layout.display = 'none'
      Reg_spacer.layout.display = 'none'
      INS_line2HBox.layout.display = 'none'
      INS_line2HBox.layout.display = 'none'

      # SUCCESS MESSAGE + CONTINUE BUTTON
      msg = Label("Done!\nYour account has been set!\nYour member ID is: " + MEM_ID)
      INS_continue_button = widgets.Button(description="Continue")

      # Group them in a container
      success_container = widgets.VBox([msg, INS_continue_button])
      display(success_container)

      # Define continue action with cleanup
      def on_continue_clicked(b):
          success_container.close()   # Remove message & button
          showPage("Home Page")

      INS_continue_button.on_click(on_continue_clicked)

  INS_register_button.on_click(on_INS_register_button_clicked)

  # layout

  INS_P_labelVBox = widgets.VBox([
      INS_FirstName_label, INS_LastName_label, INS_HomeAddress_label,
      INS_DOB_label, INS_telephone_label, INS_email_label,
      INS_password_label, INS_secQ_label, INS_secAns_label
  ])

  INS_K_labelVBox = widgets.VBox([
      INS_Kid_F_Name_label, INS_Kid_L_Name_label, INS_Kid_DOB_label
  ])

  INS_P_textboxVBox = widgets.VBox([
      INS_FirstName_textbox, INS_LastName_textbox, INS_HomeAddress_textbox,
      INS_DOB_datepicker, INS_telephone_textbox, INS_email_textbox,
      INS_password_textbox, INS_secQ_list, INS_secAns_textbox
  ])

  INS_K_textboxVBox = widgets.VBox([
      INS_Kid_F_Name_textbox, INS_Kid_L_Name_textbox, INS_Kid_DOB_datepicker
  ])

  INS_line1HBox = widgets.HBox([INS_P_labelVBox, INS_P_textboxVBox])
  INS_line2HBox = widgets.HBox([INS_K_labelVBox, INS_K_textboxVBox])
  INS_line3HBox = widgets.HBox([INS_back_button, INS_clear_button, INS_register_button])


  display(widgets.VBox([INS_line1HBox,
                        Reg_spacer,
                        INS_line2HBox,
                        INS_line3HBox
                        ]))

############# Member Register Page Tab - End #############


###########################################################
############ Member Login Page Tab - Begin ###############
###########################################################
with tb_all.output_to('Member Login Page'):

    # create 2 textboxes and 2 label
    login_error_label = Label(value='')
    login_label = Label("Login")
    id_textbox = Text(value='', placeholder='Enter a Member ID', description='Member ID:', disabled=False)
    pw_textbox = Text(description='Password:', placeholder='Enter your Password', disabled=False)

    # create 4 buttons
    login_button = widgets.Button(description="Login")
    cancel_button = widgets.Button(description="Cancel")
    forget_pw_button = widgets.Button(description="Forget Password")
    notyetregister_button = widgets.Button(description="Not yet Register?")

    def on_login_button_clicked(b):
        # clear previous error
        error_label = widgets.Label(value="")
        error_label.value = ""

        # make global variable accessible
        global current_user_id

        mem_id = id_textbox.value.strip()
        mem_pw = pw_textbox.value.strip()

        # 1. Check empty fields
        if not mem_id or not mem_pw:
            login_error_label.value = "Please fill in both fields."
            return

        # 2. Check if Member ID exists
        c.execute("SELECT * FROM MEMBER WHERE MEM_ID = '" + mem_id + "'")
        result = c.fetchone()

        if result is None:
          login_error_label.value = "Member not registered. Please register first"
          return

        db_pw = result[1]

        # 3. Check password
        if db_pw == mem_pw:
          current_user_id = mem_id
          pw_textbox.value = ""
          login_error_label.value = ""
          showPage("Main Menu")
        else:
          login_error_label.value = "Invalid Member ID or Password"

    # connect the button to this function
    login_button.on_click(on_login_button_clicked)

    def on_cancel_button_clicked(b):
        showPage("Home Page")
    cancel_button.on_click(on_cancel_button_clicked)

    def on_forget_pw_button_clicked(b):
        showPage("Forget Password Page")
    forget_pw_button.on_click(on_forget_pw_button_clicked)

    def on_notyetregister_button_clicked(b):
        showPage("Member Register Page")
    notyetregister_button.on_click(on_notyetregister_button_clicked)

    # display the objects
    member_login1HBox = widgets.HBox([login_label])
    member_login1VBox = widgets.VBox([id_textbox, pw_textbox])
    member_login2HBox = widgets.HBox([login_button, cancel_button, forget_pw_button, notyetregister_button])

    display(widgets.VBox([member_login1HBox, member_login1VBox, member_login2HBox, login_error_label]))
############# Member Login Page Tab - End ##############


##########################################################
#################### Forget password #####################
##########################################################
with tb_all.output_to('Forget Password Page'):

    # Labels + inputs
    forget_pw_label = Label("Forget your password?")
    fpw_label = Label("Registered email:     ")
    fpw_textbox = Text(value='', placeholder='Enter your registered email', disabled=False)
    fpw_enter_button = widgets.Button(description="Enter")

    spacer = Label("────────────────────────────────────────")

    security_Q_label = Label("Security question:     ")
    security_Q_textbox = Text(disabled=True) # True --> textbox is disabled, users cannot type into it

    security_ans_label = Label("Answer:     ")
    security_ans_textbox = Text(value='', placeholder='Enter your answer', disabled=True)

    security_ans_enter_button = widgets.Button(description="Enter", disabled=True)
    fpw_back_button = widgets.Button(description="Back")

    error_label = widgets.Label(value="")

    # Store member data globally for answer check
    member_row = None  # Will hold the full row when email is found

    # Show security question + enable answer field
    def on_fpw_enter_button_clicked(b):
        global member_row
        email = fpw_textbox.value.strip()

        if not email:
            error_label.value = "Please fill in your registered email."
            return

        # Clear previous
        error_label.value = ""
        security_Q_textbox.value = ""
        security_ans_textbox.value = ""
        security_ans_textbox.disabled = True
        security_ans_enter_button.disabled = True
        member_row = None

        # Query member by email
        c.execute("SELECT * FROM MEMBER WHERE MEM_EMAIL = '" + email + "'")
        row = c.fetchone()

        if row:   # If a row was returned from the database (i.e., the email exists)
            # Store full row for later answer check
            member_row = row
            # Show security question (column 9 = MEM_SECURITY_Q)
            security_Q_textbox.value = row[8]  # 0-indexed: 9th column
            # Enable answer field
            security_ans_textbox.disabled = False
            security_ans_enter_button.disabled = False
        else:
            error_label.value = "Invalid Email."

    fpw_enter_button.on_click(on_fpw_enter_button_clicked)

    # Check answer using stored row
    def on_security_ans_enter_button_clicked(b):
        global member_row
        answer = security_ans_textbox.value.strip()

        # Clear previous
        error_label.value = ""

        if not answer:
            error_label.value = "Please fill in your answer."
            return

        if member_row and answer == member_row[9]:  # 10th column = MEM_SECURITY_ANS
            showPage("Reset Password Page")
        else:
            error_label.value = "Wrong answer, please enter again."

    security_ans_enter_button.on_click(on_security_ans_enter_button_clicked)

    def on_fpw_back_button_clicked(b):
        showPage("Member Login Page")
    fpw_back_button.on_click(on_fpw_back_button_clicked)

    # Layout
    security_labelVBox = widgets.VBox([security_Q_label, security_ans_label])
    security_textboxVBox = widgets.VBox([security_Q_textbox, security_ans_textbox])
    fpw_line1HBox = widgets.HBox([fpw_label, fpw_textbox])
    fpw_line2HBox = widgets.HBox([security_labelVBox, security_textboxVBox])
    fpw_line3HBox = widgets.HBox([fpw_enter_button, fpw_back_button])

    # Display
    display( widgets.VBox([ fpw_line1HBox, fpw_line3HBox,
        ]),
        error_label,
        spacer,
        widgets.HBox([fpw_line2HBox]),
        security_ans_enter_button
    )

############# Forget Password Page Tab - End ##############

###########################################################
################## Update Personal Info ###################
###########################################################
with tb_all.output_to('Personal Info'):  # when I click edit my profile I was directed to Personal Info tab from main menu

    # Title
    View_info_page_label = Label("View/Edit My Profile")

    # Load existing information for the given Member ID as member already login
    def load_member_profile_information():
        global current_user_id
        c.execute(
            "SELECT MEM_ADD, TO_CHAR(MEM_DOB,'YYYY-MM-DD'), MEM_TEL, MEM_EMAIL, MEM_PW "
            "FROM MEMBER WHERE MEM_ID = '" + current_user_id + "'"
        )

        for row in c:   # Loop through each row returned by the database query
            txtAddress.value = row[0]
            DOB_datepicker.value = datetime.strptime(row[1], "%Y-%m-%d").date() if row[1] else None
            txtTel.value = row[2]
            txtEmail.value = row[3]
            txtPW.value = row[4]

    # Buttons
    back_main_button = widgets.Button(description="Back to Main Menu")
    Edit_member_profile_button = widgets.Button(description="Confirm Edit")

    # Prompt for user input of new updated profile information
    lblAddr = Label("Home Address:")
    txtAddress = widgets.Text(placeholder="Enter new address")

    lblDOB = Label("DOB (YYYY-MM-DD):")
    DOB_datepicker = DatePicker(placeholder="DOB (YYYY-MM-DD):", disabled=False)  # disabled = false will enable the datepicker

    lblTel = Label("Telephone:")
    txtTel = widgets.Text(placeholder="Enter new telephone")

    lblEmail = Label("Email:")
    txtEmail = widgets.Text(placeholder="Enter new email")

    lblPW = Label("Password:")
    txtPW = widgets.Text(placeholder="Enter new password")


    def on_back_main_button_clicked(b):
        showPage("Main Menu")

    # message label used to display successful edit
    confirm_output_label = Label(value='')

    def on_Edit_member_profile_button_clicked(b):
        c.execute(
            """
            UPDATE MEMBER
            SET MEM_ADD = '{}',
                MEM_DOB = TO_DATE('{}', 'YYYY-MM-DD'),
                MEM_TEL = '{}',
                MEM_EMAIL = '{}',
                MEM_PW = '{}'
            WHERE MEM_ID = '{}'
            """.format(
                txtAddress.value.strip(),
                DOB_datepicker.value.strftime('%Y-%m-%d'),
                txtTel.value.strip(),
                txtEmail.value.strip(),
                txtPW.value.strip(),
                current_user_id
            )
        )
        c.execute("commit")
        confirm_output_label.value = "Your profile has been successfully updated."

    back_main_button.on_click(on_back_main_button_clicked)
    Edit_member_profile_button.on_click(on_Edit_member_profile_button_clicked)

    # Layout
    # Arrange labels and inputs in two vertical boxes side by side
    member_labelVBox = widgets.VBox([lblAddr, lblDOB, lblTel, lblEmail, lblPW])
    member_inputVBox = widgets.VBox([txtAddress, DOB_datepicker, txtTel, txtEmail, txtPW])

    # Combine the label and input columns horizontally
    member_line1HBox = widgets.HBox([member_labelVBox, member_inputVBox])

    # Buttons line
    member_line2HBox = widgets.HBox([back_main_button, Edit_member_profile_button])

    # Display layout
    display(View_info_page_label, member_line1HBox, member_line2HBox, confirm_output_label)
############# Update Personal Info Tab - End ##############

##########################################################
############ Reset Password Page Tab - Begin ###############
##########################################################
with tb_all.output_to('Reset Password Page'):

  # Labels + inputs
  reset_pw_label = Label("Reset your password")

  rp_new_pw_label = Label("New Password:     ")
  rp_new_pw_textbox = Text(value='', placeholder='Enter new password', disabled=False)

  rp_re_pw_label = Label("Re-enter New Password:     ")
  rp_re_pw_textbox = Text(value='', placeholder='Re-enter new password', disabled=False)

  # create buttons
  rp_back_button = widgets.Button(description="Back")
  rp_confirm_button = widgets.Button(description="Confirm")

  error_label = widgets.Label(value="")

  # Actions
  def on_rp_confirm_button_clicked(b):
    error_label.value = ""

    new_pw = rp_new_pw_textbox.value.strip()
    re_pw  = rp_re_pw_textbox.value.strip()

    if not new_pw or not re_pw:
        error_label.value = "Please fill in both password fields."
        return

    if new_pw != re_pw:
        error_label.value = "Passwords do not match."
        return

    # Retrieve email from Forget Password Page (still in scope)
    email = fpw_textbox.value.strip()

    if not email:
        error_label.value = "Email not found. Please go back and re-enter."
        return

    # Fetch current password (do NOT display it)
    c.execute("SELECT MEM_PW FROM MEMBER WHERE mem_email = '" + email + "'")
    row = c.fetchone()

    if not row:
        error_label.value = "Invalid email or account not found."
        return

    current_pw = row[0]

    if new_pw == current_pw:
        error_label.value = "New password must be different from current password."
        return

    # Update password
    c.execute("UPDATE MEMBER SET MEM_PW = '" + new_pw + "' WHERE mem_email = '" + email + "'")
    c.execute("commit")

    error_label.value = "Password updated successfully!"
    rp_confirm_button.disabled = True
    rp_new_pw_textbox.disabled = True
    rp_re_pw_textbox.disabled = True
  rp_confirm_button.on_click(on_rp_confirm_button_clicked)


  def on_rp_back_button_clicked(b):
      showPage("Forget Password Page")
  rp_back_button.on_click(on_rp_back_button_clicked)

  # Layout
  rp_labelVBox = widgets.VBox([rp_new_pw_label,rp_re_pw_label])
  rp_textboxVBox = widgets.VBox([rp_new_pw_textbox,rp_re_pw_textbox])
  rp_line1HBox = widgets.HBox([rp_labelVBox, rp_textboxVBox])
  rp_line2HBox = widgets.HBox([rp_back_button, rp_confirm_button])

  display(widgets.VBox([
      rp_line1HBox,
      rp_line2HBox,
      error_label
  ]))
############# Reset Password Page Tab - End ##############


##########################################################
############## Edit My Kid Information ###################
##########################################################

with tb_all.output_to('My Kid Information'):

    kid_label = Label("View/Edit My Kid Information")

    # Load existing information for the given Member ID's kid as member already login
    def load_kid_information():
        global current_user_id

        c.execute("SELECT K_FNAME, K_LNAME, TO_CHAR(K_DOB,'YYYY-MM-DD') "
                  "FROM MEMBER WHERE MEM_ID = '" + current_user_id + "'")

        for row in c:
            txtK_FNAME.value = row[0]
            txtK_LNAME.value = row[1]
            DOB_kid_datepicker.value = datetime.strptime(row[2], "%Y-%m-%d").date() if row[2] else None

    # Buttons
    back_main_button = widgets.Button(description="Back to Main Menu")
    Edit_kid_info_button = widgets.Button(description="Confirm Edit")
    lblMsg = Label("")

    # Prompt for user input of new updated kid information
    lblK_FNAME = Label("Kid's First Name:")
    txtK_FNAME = widgets.Text(placeholder="Enter first name")

    lblK_LNAME = Label("Kid's Last Name:")
    txtK_LNAME = widgets.Text(placeholder="Enter last name")

    lblK_DOB = Label("Kid's DOB:")
    DOB_kid_datepicker = DatePicker(placeholder="YYYY-MM-DD", disabled=False)

    # Button Event Handlers
    def on_back_main_button_clicked(b):
        showPage("Main Menu")

    def on_btnSaveKidInfo_clicked(b):
        global current_user_id
        c.execute(
            "UPDATE MEMBER SET K_FNAME = '{}', K_LNAME = '{}', K_DOB = TO_DATE('{}','YYYY-MM-DD') "
            "WHERE MEM_ID = '{}'".format(
                txtK_FNAME.value.strip(),
                txtK_LNAME.value.strip(),
                DOB_kid_datepicker.value.strftime('%Y-%m-%d'),
                current_user_id
            )
        )
        c.execute("commit")
        lblMsg.value = " Kid Information Updated Successfully!"

    Edit_kid_info_button.on_click(on_btnSaveKidInfo_clicked)
    back_main_button.on_click(on_back_main_button_clicked)

    # Layout
    kid_labelVBox = widgets.VBox([lblK_FNAME, lblK_LNAME, lblK_DOB])
    kid_inputVBox = widgets.VBox([txtK_FNAME, txtK_LNAME, DOB_kid_datepicker])
    kid_line1HBox = widgets.HBox([kid_labelVBox, kid_inputVBox])
    kid_line2HBox = widgets.HBox([back_main_button, Edit_kid_info_button])

    display(widgets.VBox([
        kid_label,
        kid_line1HBox,
        kid_line2HBox,
        lblMsg
    ]))

########################################################### MANAGER BELOW ###################################################################


##########################################################
################ Manager Login Tab - Begin ################
##########################################################
with tb_all.output_to('Manager Login Page'):
    # Create widgets
    title_label = Label("Manager Login")

    manager_id = Text(placeholder="Enter your assigned ID", description="Manager ID:")
    manager_password = Text(placeholder="Enter your password", description="Password:")
    error_msg = Label("Invalid Manager ID or Password", style={'color': 'red'})
    error_msg.layout.visibility = 'hidden'

    login_btn = widgets.Button(description="Login", button_style='primary')
    back_btn = widgets.Button(description="Back")

    # Event handlers
    def on_login_click(b):
      # clear previous error
      error_msg.layout.visibility = 'hidden'

      man_id = manager_id.value.strip()
      man_pw = manager_password.value.strip()

      if not man_id or not man_pw:
        error_msg.value = "Please fill in both fields."
        error_msg.layout.visibility = 'visible'
        return


      c.execute("SELECT * FROM MANAGER WHERE MAN_ID = '" + man_id + "' AND MAN_PW = '" + man_pw + "'")

      # Check existence with loop
      found = False
      for row in c:
          found = True
          break  # no need for more, since we just check if match exists

      if found:                     # match found
          error_msg.layout.visibility = 'hidden' # clear error on success
          showPage("Manager Dashboard")
      else:
          error_msg.value = "Invalid Manager ID or Password."
          error_msg.layout.visibility = 'visible'


    def on_back_click(b):
        showPage("Home Page")

    login_btn.on_click(on_login_click)
    back_btn.on_click(on_back_click)

    # Layout
    manager_login1HBox = widgets.HBox([title_label])
    manager_login2VBox = widgets.VBox([manager_id, manager_password])
    manager_login2HBox = widgets.HBox([login_btn, back_btn])


    display(widgets.VBox([manager_login1HBox, manager_login2VBox, manager_login2HBox, error_msg]))
################ Manager Login Tab - End ################



###############################################################
################ Manager Dashboard Tab - Begin ################
##############################################################
with tb_all.output_to('Manager Dashboard'):
    # Create widgets
    title_label = Label("Manager Dashboard")

    logout_btn = widgets.Button(description="Logout")

    teachers_label = Label("Teacher Accounts")
    add_teacher_btn = widgets.Button(description="Add New Teacher")
    edit_teacher_btn = widgets.Button(description="Edit Teacher")
    delete_teacher_btn = widgets.Button(description="Delete Teacher")

    courses_label = Label("Course Details")
    add_course_btn = widgets.Button(description="Add New Course")
    edit_course_btn = widgets.Button(description="Edit Course")
    delete_course_btn = widgets.Button(description="Delete Course")
    refresh_btn = widgets.Button(description = 'Refresh the Page')

    # Create tables (simplified as text areas for demo)
    teachers_table = widgets.Textarea(description="Teachers:", disabled=True, layout={'width': '100%', 'height': '200px'})
    courses_table = widgets.Textarea(description="Courses:", disabled=True, layout={'width': '100%', 'height': '300px'})

    def update_tables():
        # Update teachers table
        #teachers_text = "ID\tName\t\t\tEmail\t\t\t\tStatus\n"
        #teachers_text += "-" * 80 + "\n"

        connection_url = f"oracle+oracledb://{USERNAME}:{PASSWORD}@{HOST_NAME}:{PORT_NUMBER}/?service_name={SERVICE_NAME}"
        engine = create_engine(connection_url)

        sql_query = "SELECT * FROM TEACHER"
        df = pd.read_sql(sql_query, engine)
        #print(df.columns.tolist())
        df_selected = df[['ter_id', 'ter_lname', 'ter_fname','ter_email','ter_status']]
        df_selected = df_selected.rename(columns={'ter_id': 'ID', "ter_lname":"Last Name", "ter_fname": "First Name","ter_email" : "Email","ter_status": "Status" })
        #out = Output()

        #for index in range(len(df)):
          #teachers_text += f"{df.iloc[index,0]}\t{df.iloc[index, 1]} {df.iloc[index, 2]}\t{df.iloc[index, 3]}\t{df.iloc[index, 5]}\n"
        #teachers_table.value = teachers_text
        teachers_table.value = df_selected.to_markdown(index = False)
        # Update courses table

        #out = Output()

        #courses_text = "ID\tName\t\t\tDate\t   Time\t\tTeacher\t Age\tQuota\n"
        #courses_text += "-" * 80 + "\n"
        sql_query1 = "SELECT * FROM COURSES"
        df_course = pd.read_sql(sql_query1, engine)
        #df_course_selected = df_course[["cou_id", "cou_name", "cou_date", "teacher_ter_id", "cou_age_min","cou_age_max", "cou_quota"]]

        df_course["cou_age_range"] = (
            df_course["cou_age_min"].astype(str) + "-" +
            df_course["cou_age_max"].astype(str)
        )

        # inplace = False --> Return a NEW DataFrame, don't change the original
        df_course =  df_course.drop(columns=["cou_age_min", "cou_age_max","cou_desc","cou_enrollment_deadline","cou_remarks","cou_category"], inplace=False)

        df_course = df_course.rename(columns={"cou_id":"ID", "cou_name":"Name", "cou_date":"Date", "teacher_ter_id":"Teacher", "cou_age_range" : "Age", "cou_quota":"Quota", "cou_status" : "Status"})

        #for index1 in range(len(df_course)):

          #courses_text += f"{df_course.iloc[index1, 0]}\t{df_course.iloc[index1, 1]}\t{df_course.iloc[index1, 4]}\t" \
          #f"{df_course.iloc[index1, 11]}\t {df_course.iloc[index1, 8]}" \
          #f"-{df_course.iloc[index1, 7]}\t{df_course.iloc[index1, 3]}\n"

        #courses_table.value = courses_text
        courses_table.value = df_course.to_markdown(index = False)

    # Event handlers
    def on_logout_click(b):
        showPage("Home Page")
    logout_btn.on_click(on_logout_click)

    def on_refresh_btn_click(b):
        update_tables()
    refresh_btn.on_click(on_refresh_btn_click)

    def on_add_teacher_click(b):
        showPage("Add Teacher")
    add_teacher_btn.on_click(on_add_teacher_click)

    def on_edit_teacher_click(b):
        showPage("Edit Teacher")
    edit_teacher_btn.on_click(on_edit_teacher_click)

    def on_delete_teacher_click(b):
        showPage("Delete Teacher")
    delete_teacher_btn.on_click(on_delete_teacher_click)

    def on_add_course_click(b):
        showPage("Add Course")
    add_course_btn.on_click(on_add_course_click)

    def on_edit_course_click(b):
        showPage("Edit Course")
    edit_course_btn.on_click(on_edit_course_click)

    def on_delete_course_click(b):
        showPage("Delete Course")
    delete_course_btn.on_click(on_delete_course_click)


    # Initial table update / keep updating table
    update_tables()


    # Layout
    manager_db1HBox = widgets.HBox([logout_btn, refresh_btn])
    manager_db1VBox = widgets.VBox([title_label, teachers_label])
    manager_db2HBox = widgets.HBox([add_teacher_btn, edit_teacher_btn, delete_teacher_btn])
    manager_db2VBox = widgets.VBox([teachers_table, courses_label])
    manager_db3HBox = widgets.HBox([add_course_btn, edit_course_btn, delete_course_btn])
    manager_db3VBox = widgets.VBox([courses_table])


    display(widgets.VBox([manager_db1HBox, manager_db1VBox, manager_db2HBox, manager_db2VBox,
                          manager_db3HBox, manager_db3VBox, error_msg]))
################ Manager Dashboard Tab - End ################



##########################################################
################ Add Teacher Tab - Begin #################
##########################################################
with tb_all.output_to('Add Teacher'):

    # Label & Textbox
    Teacher_title_label = Label("Add New Teacher")

    F_Name_label = Label("First Name: ")
    F_Name_textbox = Text(placeholder= 'First_Name', disable = False)

    S_Name_label = Label("Surname Name: ")
    S_Name_textbox = Text(placeholder= 'Surname', disable = False)

    Email_label = Label("Email: ")
    Email_textbox = Text(placeholder = "Email", disable = False)

    PW_label = Label("Password: ")
    PW_textbox = Text(placeholder = "Password", disable = False)

    Status_label = Label("Status: ")
    Status_cmb = widgets.Dropdown(options=['Active', 'Not Active', 'On Leave', 'Training', 'Probation'], value = None)

    Tea_message_label = Label("")

    Tea_add_button = widgets.Button(description = 'Add')
    Tea_clear_button = widgets.Button(description = 'Clear')
    Tea_back_button = widgets.Button(description = 'Back')


    # Add Teacher
    def on_Tea_add_button_clicked(b):

      Tea_required_field = [
          F_Name_textbox.value.strip(),
          S_Name_textbox.value.strip(),
          Email_textbox.value.strip(),
          PW_textbox.value.strip(),
          Status_cmb.value
                            ]

      if not all(Tea_required_field):
          Tea_message_label.value = "Missing data: Please fill in all required fields."
          return  # Stop — don't add teacher

      # generate next Txxx ID
      c.execute("SELECT TER_ID FROM TEACHER ORDER BY TER_ID DESC")
      row = c.fetchone()
      if row is None:
            TER_ID = "T001"
      else:
            TER_last_id = row[0]
            TER_num = int(TER_last_id[1:]) + 1
            TER_ID = f"T{TER_num:03d}"

      c.execute(
          """Insert into TEACHER Values ('{}','{}', '{}', '{}', '{}','{}' )
          """.format(TER_ID, F_Name_textbox.value, S_Name_textbox.value, Email_textbox.value, PW_textbox.value, Status_cmb.value))
      c.execute("commit")
      Tea_add_button.disabled = True
      Tea_message_label.value = "Done!"
    Tea_add_button.on_click(on_Tea_add_button_clicked)

    # Clear
    def on_Tea_clear_button_clicked(b):
      F_Name_textbox.value = ""
      S_Name_textbox.value = ""
      Email_textbox.value = ""
      PW_textbox.value = ""
      Status_cmb.value = None
      Tea_add_button.disabled = False
      Tea_message_label.value = ""
    Tea_clear_button.on_click(on_Tea_clear_button_clicked)

    # Back
    def on_Tea_back_button_clicked(b):
      showPage("Manager Dashboard")
    Tea_back_button.on_click(on_Tea_back_button_clicked)

    # Display
    Tea_labelVBox = widgets.VBox([ F_Name_label, S_Name_label, Email_label, PW_label, Status_label])
    Tea_textboxVBox = widgets.VBox([ F_Name_textbox, S_Name_textbox, Email_textbox, PW_textbox, Status_cmb])
    Tea_line1HBox = widgets.HBox([Tea_labelVBox, Tea_textboxVBox])
    Tea_line2HBox = widgets.HBox([Tea_add_button, Tea_clear_button, Tea_back_button])


    display(widgets.VBox([Teacher_title_label, Tea_line1HBox, Tea_message_label, Tea_line2HBox]))
################ Add Teacher Tab - End ################



##########################################################
################ Edit Teacher Tab - Begin ################
##########################################################
with tb_all.output_to('Edit Teacher'):

    # Create widgets for teacher ID retrieval
    ET_title_label = Label("EDIT TEACHER INFORMATION") # Title label

    ET_teacher_id_input = Text(placeholder="Enter Teacher ID (e.g., T001)", description="Teacher ID:", layout={'width': '400px'})                      # Input field for teacher ID
    ET_retrieve_btn = widgets.Button(description="Retrieve Teacher", button_style='primary', layout={'width': '200px'})                                # Button to trigger teacher data retrieval

    spacer = Label("-----------------------------------------------------------------------------------------------------------------------------------") # Visual separator

    ET_current_header = Label("CURRENT INFORMATION", style={'font_weight': 'bold'}, layout={'width': '350px'})                                        # Header for current information section
    ET_edit_header = Label("MAKE CHANGES", style={'font_weight': 'bold'}, layout={'width': '350px'})                                                  # Header for editable fields section

    ET_lname_display = Text(description="Last Name:", disabled=True, layout={'width': '350px'})
    ET_lname_edit = Text(placeholder="Enter new last name", description="Last Name:", layout={'width': '350px'})

    ET_fname_display = Text(description="First Name:", disabled=True, layout={'width': '350px'})
    ET_fname_edit = Text(placeholder="Enter new first name", description="First Name:", layout={'width': '350px'})

    ET_email_display = Text(description="Email:", disabled=True, layout={'width': '350px'})
    ET_email_edit = Text(placeholder="Enter new email", description="Email:", layout={'width': '350px'})

    ET_status_display = Text(
        description="Status:",
        disabled=True,
        layout={'width': '350px'}
    )
    ET_status_edit = widgets.Dropdown(
        options=['Active', 'Not Active', 'On Leave', 'Training', 'Probation'],
        description="Status:",
        layout={'width': '350px'}
    )

    ET_password_label = Label("New Password:", layout={'width': '350px'})
    ET_password_edit = Text(placeholder="Leave blank to keep current password",
                           description="New Password:", layout={'width': '350px'})

    ET_message_label = Label("")                                                                                                                   # For displaying success/error messages to Manager

    # Buttons
    ET_clear_btn = widgets.Button(description="Clear Form", button_style='warning', layout={'width': '150px'})                                        #  Clear all form fields and reset the interface
    ET_edit_btn = widgets.Button(description="Update Teacher", button_style='success', layout={'width': '150px'})                                     #  Save all changes made to teacher information
    ET_back_btn = widgets.Button(description="Back to Dashboard", layout={'width': '150px'})                                                          #  Navigate back to main dashboard

    # Retrieve teacher function
    def on_ET_retrieve_btn_click(b):
        ET_teacher_id =  ET_teacher_id_input.value.strip()                                                                                             # Get and clean the teacher ID input, removing any extra spaces

        # Validation
        if not ET_teacher_id:
            ET_message_label.value = "ERROR: Please enter a Teacher ID"                                                                                  # Validation: Check if teacher ID is provided
            return # Stop if validation fails

        try:
            # Retrieve teacher from database
            c.execute("SELECT TER_ID, TER_LNAME, TER_FNAME, TER_EMAIL, TER_PW, TER_STATUS FROM TEACHER WHERE TER_ID = '" +  ET_teacher_id + "'")
            row = c.fetchone()                                                                                                                            #   Retrieve teacher data from database using teacher ID

            if row: # Check if teacher was found in database
                # Display current information
                ET_lname_display.value = row[1]
                ET_fname_display.value = row[2]
                ET_email_display.value = row[3]
                ET_status_display.value = row[5]

                # Pre-fill edit fields with current values
                ET_lname_edit.value = row[1]
                ET_fname_edit.value = row[2]
                ET_email_edit.value = row[3]
                ET_status_edit.value = row[5]
                ET_password_edit.value = ''

                ET_message_label.value = f"SUCCESS: Teacher { ET_teacher_id} retrieved successfully"     # Success message confirming retrieval
            else:
                ET_message_label.value = f"ERROR: No teacher found with ID: { ET_teacher_id}"            # Error message for non-existent teacher ID

                # Clear all fields
                ET_lname_display.value = ""
                ET_fname_display.value = ""
                ET_email_display.value = ""
                ET_status_display.value = "Active"
                ET_lname_edit.value = ""
                ET_fname_edit.value = ""
                ET_email_edit.value = ""
                ET_status_edit.value = "Active"
                ET_password_edit.value = ""

        except Exception as e:                                              # Handle any database errors that occured
            ET_message_label.value = f"ERROR: Database error - {str(e)}"
    ET_retrieve_btn.on_click(on_ET_retrieve_btn_click)

    # Clear function
    def on_clear_click(b):
        ET_teacher_id_input.value = ""
        ET_lname_display.value = ""
        ET_fname_display.value = ""
        ET_email_display.value = ""
        ET_status_display.value = "Active"
        ET_lname_edit.value = ""
        ET_fname_edit.value = ""
        ET_email_edit.value = ""
        ET_status_edit.value = "Active"
        ET_password_edit.value = ""
        ET_message_label.value = ""
    ET_clear_btn.on_click(on_clear_click)

    # Edit teacher function
    def on_ET_edit_btn_click(b):
        ET_teacher_id =  ET_teacher_id_input.value.strip()

        # Validation
        if not ET_teacher_id:
            ET_message_label.value = "ERROR: Please enter a Teacher ID first"
            return

        # Validate required fields
        if not all([ET_lname_edit.value, ET_fname_edit.value, ET_email_edit.value]):
            ET_message_label.value = "ERROR: Please fill in all required fields (Last Name, First Name, Email)"
            return

        try:
            # Check if teacher exists
            c.execute("SELECT TER_ID FROM TEACHER WHERE TER_ID = '" + ET_teacher_id + "'")
            if not c.fetchone():
                ET_message_label.value = f"ERROR: No teacher found with ID: {ET_teacher_id}"
                return

            # Update teacher in database
            if ET_password_edit.value:
                # Update all fields including password
                c.execute(
                    """UPDATE TEACHER SET TER_LNAME = '{}', TER_FNAME = '{}', TER_EMAIL = '{}', TER_STATUS = '{}', TER_PW = '{}' WHERE TER_ID = '{}'""".format(
                        ET_lname_edit.value, ET_fname_edit.value, ET_email_edit.value, ET_status_edit.value, ET_password_edit.value, ET_teacher_id
                    )
                )
            else:
                # Update all fields except password
                c.execute(
                    """UPDATE TEACHER SET TER_LNAME = '{}', TER_FNAME = '{}', TER_EMAIL = '{}', TER_STATUS = '{}' WHERE TER_ID = '{}'""".format(
                        ET_lname_edit.value, ET_fname_edit.value, ET_email_edit.value, ET_status_edit.value, ET_teacher_id
                    )
                )

            c.execute("COMMIT")

            # Update display fields with new values
            ET_lname_display.value = ET_lname_edit.value
            ET_fname_display.value = ET_fname_edit.value
            ET_email_display.value = ET_email_edit.value
            ET_status_display.value = ET_status_edit.value

            ET_message_label.value = f"SUCCESS: Teacher {ET_teacher_id} was updated successfully"

            # Clear password field
            ET_password_edit.value = ""

        except Exception as e:  # Handle any errors that occured
            ET_message_label.value = f"ERROR: Failed to update teacher - {str(e)}"
    ET_edit_btn.on_click(on_ET_edit_btn_click)

    # Back to dashboard function
    def on_ET_back_btn_click(b):
        showPage("Manager Dashboard")
    ET_back_btn.on_click(on_ET_back_btn_click)

    # Layout
    TeaEdit_line1HBox = widgets.HBox([ET_teacher_id_input, ET_retrieve_btn]) # Horizontal layout: Teacher ID input and retrieve button side by side
    TeaEdit_line1VBox = widgets.VBox([ET_current_header, ET_lname_display, ET_fname_display, ET_email_display, ET_status_display])  # Vertical layout: Current information display fields stacked vertically
    TeaEdit_line2VBox = widgets.VBox([ET_edit_header, ET_lname_edit, ET_fname_edit, ET_email_edit, ET_status_edit, ET_password_edit])  # Vertical layout: Editable fields stacked vertically for making changes
    TeaEdit_line2HBox = widgets.HBox([ET_clear_btn, ET_edit_btn, ET_back_btn]) # Horizontal layout: Action buttons arranged side by side
    TeaEdit_line3HBox = widgets.HBox([TeaEdit_line1VBox, TeaEdit_line2VBox])  # Horizontal layout: Current info and edit sections displayed side by side for comparison

    display(widgets.VBox([
        ET_title_label,
        TeaEdit_line1HBox,
        spacer,
        TeaEdit_line3HBox,
        TeaEdit_line2HBox,
        ET_message_label
    ]))  # Final vertical arrangement of all UI components
################ Edit Teacher Tab - End ################


##########################################################
################ Delete Teacher Tab - Begin ################
##########################################################
with tb_all.output_to('Delete Teacher'):
    # create labels and inputs
    title_label = Label("Delete Teacher")

    DEL_id_label = Label("Teacher ID: ")
    DEL_id_textbox = Text(placeholder="Teacher ID")

    DEL_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

    DEL_L_name_textbox = Text(placeholder="Name will appear here", description="Last Name:", disabled=True)
    DEL_F_name_textbox = Text(placeholder="Name will appear here", description="First Name:", disabled=True)
    DEL_email_textbox = Text(placeholder="Email will appear here", description="Email:", disabled=True)

    status = widgets.Dropdown(
        options=['Active', 'Not Active', 'On Leave', 'Training', 'Probation'],
        value='Active',
        description="Status:"
    )

    DEL_Tea_message_label = Label("")

    # create 4 buttons
    DEL_retrieve_button = widgets.Button(description="Retrieve")
    DEL_delete_button = widgets.Button(description="Delete")
    DEL_clear_button = widgets.Button(description="Clear")
    DEL_back_button = widgets.Button(description="Back")

    def on_DEL_retrieve_button_clicked(b):
        c.execute("SELECT * FROM TEACHER WHERE TER_ID = :1", (DEL_id_textbox.value,))
        for row in c:
            DEL_L_name_textbox.value = row[1]
            DEL_F_name_textbox.value = row[2]
            DEL_email_textbox.value = row[3]
            status.value = row[5]

    DEL_retrieve_button.on_click(on_DEL_retrieve_button_clicked)

    def on_DEL_delete_button_clicked(b):
        teacher_id = DEL_id_textbox.value.strip()

        if not teacher_id:
            DEL_Tea_message_label.value = "Please enter a Teacher ID first."
            return

        try:
            # Check if teacher exists in COURSES table (has dependent records)
            c.execute("SELECT COUNT(*) FROM COURSES WHERE TEACHER_TER_ID = :1", (teacher_id,))
            course_count = c.fetchone()[0]    # Course ID locates at position [0] in database

            if course_count > 0:
                DEL_Tea_message_label.value = f"Cannot delete teacher {teacher_id}. This teacher is assigned to {course_count} course(s). Please reassign or delete those courses first."
                return

            # If no dependent courses, proceed with deletion
            c.execute("DELETE FROM TEACHER WHERE TER_ID = :1", (teacher_id,))     # Run a SQL query to find a teacher by ID, using a SAFE placeholder :1 (First Parameter) and passing the actual value separately
            conn.commit()  # Use conn.commit() instead of c.execute("commit") since c.execute("commit") is to run SQL instead of saving changes
            DEL_Tea_message_label.value = "Delete Success!"

            # DO NOT clear the form - keep the success message visible
            # The user can see the success message until they click "Back to Dashboard" or "Clear"

        except Exception as e:
            DEL_Tea_message_label.value = f"Error: {e}"

    DEL_delete_button.on_click(on_DEL_delete_button_clicked)

    def on_DEL_back_button_clicked(b):
        showPage("Manager Dashboard")
    DEL_back_button.on_click(on_DEL_back_button_clicked)

    # display the objects
    DEL_line1HBox = widgets.HBox([title_label])
    DEL_line2HBox = widgets.HBox([DEL_id_label, DEL_id_textbox, DEL_retrieve_button])
    DEL_line3HBox = widgets.HBox([DEL_spacer])
    DEL_textboxVBox = widgets.VBox([DEL_L_name_textbox, DEL_F_name_textbox, DEL_email_textbox, status])
    DEL_line4HBox = widgets.HBox([DEL_textboxVBox])
    DEL_line5HBox = widgets.HBox([DEL_delete_button, DEL_clear_button])
    DEL_line6HBox = widgets.HBox([DEL_back_button])

    display(widgets.VBox([DEL_line1HBox, DEL_line2HBox, DEL_line3HBox, DEL_line4HBox, DEL_Tea_message_label, DEL_line5HBox, DEL_line6HBox]))
################ Delete Teacher Tab - End ################


##########################################################
################ Add Course Tab - Begin ################
##########################################################
with tb_all.output_to('Add Course'):

# Label & Textboxs
  COU_title_label = Label("Add New Course")

  COU_Name_textbox = Text(description = 'Name: ', placeholder= 'Input course name', disable = False)

  COU_DESC_textbox = widgets.Textarea(description = 'Decription: ', placeholder= 'Input course description (max 500 characters)', layout={'width': '500px', 'height':'100px'})

  COU_TER_textbox = Text(description = "Teacher: ",placeholder = "", disable = False)

  COU_TER_ID_textbox = Text(description = 'Teacher ID: ', placeholder = "", disable = False)

  COU_CAT_cmb = widgets.Dropdown(description = 'Category: ', options=['Art', 'Math', 'Science', 'Music', 'Dance', 'Sports',
                'Language', 'Coding', 'Cooking', 'Drama', 'Yoga', 'Robotics'], value= None)

  COU_AGE_MIN_intbox = IntText(description= "Min Age: ", disable = False)

  COU_AGE_MAX_intbox = IntText(description= "Max Age: ", disable = False)

  COU_QUOTA_intbox = IntText(description = "Quota: ", disable = False)

  COU_STATUS_cmb = widgets.Dropdown(description = 'Status: ', options=['Active', 'Cancelled', 'Postponed', 'Full', 'Upcoming'],
        value='Active')

  COU_DATE_datepicker = DatePicker(description = 'Course Date: ', placeholder="", disable = False)

  COU_ENT_DATE_datepicker = DatePicker(description = "Enrolment Date: ", placeholder="", disable = False)

  COU_REMARKS_textbox = widgets.Textarea(description = "Remarks: ", placeholder = "Optional (max 300 characters)", layout={'width': '500px', 'height':'100px'} )

  time_options = [f"{h:02d}:{m:02d}" for h in range(7,21) for m in (0, 30)]

  COU_START_TIME_cmb = widgets.Dropdown(description='Start Time:', options=time_options, value=None)

  COU_END_TIME_cmb = widgets.Dropdown(description='End Time:', options=time_options, value=None)

  COU_message_label = Label("")

# Create buttons

  COU_add_button = widgets.Button(description= "Add Course")
  COU_cancel_button = widgets.Button(description= "Clear")
  COU_back_button = widgets.Button(description= "Back")


# Event Handler

  def on_COU_add_button_clicked(b):
    COU_required_field = [
        COU_Name_textbox.value.strip(),
        COU_DESC_textbox.value.strip(),
        COU_QUOTA_intbox.value,
        COU_DATE_datepicker.value,
        COU_ENT_DATE_datepicker.value,
        COU_STATUS_cmb.value,
        COU_CAT_cmb.value,
        COU_TER_ID_textbox.value.strip(),
        COU_START_TIME_cmb.value,
        COU_END_TIME_cmb.value
       ]

    # validate if all fields are filled
    if not all(COU_required_field):
        COU_message_label.value = "Missing data: Please fill in all the fields."
        return  # Stop execution — don't add course

    # age valiadation
    if COU_AGE_MIN_intbox.value > COU_AGE_MAX_intbox.value:
        COU_message_label.value = "ERROR: Min age > Max age !"
        return # Stop execution - mix age > max age

    # time validation
    if COU_START_TIME_cmb.value >= COU_END_TIME_cmb.value:
        COU_message_label.value = "ERROR: Start time must be earlier than End time!"
        return


    # generate next COUxxx ID
    c.execute("SELECT COU_ID FROM COURSES ORDER BY COU_ID DESC")
    row = c.fetchone()
    if row is None:
          COU_ID = "C001"
    else:
        COU_last_id = row[0]
        COU_num = int(COU_last_id[1:]) + 1
        COU_ID = f"C{COU_num:03d}"

    c.execute(
        """Insert into COURSES Values (
          '{}',
          '{}',
          '{}',
          {},
          TO_DATE('{}','YYYY-MM-DD'),
          TO_DATE('{}','YYYY-MM-DD'),
          '{}',
          {},
          {},
          '{}',
          '{}',
          '{}',
          '{}',
          '{}'
          )

        """.format(
            COU_ID,
            COU_Name_textbox.value,
            COU_DESC_textbox.value,
            COU_QUOTA_intbox.value,
            COU_DATE_datepicker.value.strftime('%Y-%m-%d'),
            COU_ENT_DATE_datepicker.value.strftime('%Y-%m-%d'),
            COU_STATUS_cmb.value,
            COU_AGE_MAX_intbox.value if COU_AGE_MAX_intbox.value else 'NULL',
            COU_AGE_MIN_intbox.value if COU_AGE_MIN_intbox.value else 'NULL',
            COU_REMARKS_textbox.value,
            COU_CAT_cmb.value,
            COU_TER_ID_textbox.value,
            COU_START_TIME_cmb.value,   # ← Now stored as '07:30' (VARCHAR2)
            COU_END_TIME_cmb.value      # ← Now stored as '14:00' (VARCHAR2)
            )
        )
    c.execute("commit")
    COU_add_button.disabled = True
    COU_message_label.value = "Done !"
  COU_add_button.on_click(on_COU_add_button_clicked)

  # Clear
  def on_COU_clear_button_clicked(b):
    COU_Name_textbox.value = ""
    COU_DESC_textbox.value = ""
    COU_TER_textbox.value = ""
    COU_TER_ID_textbox.value = ""
    COU_CAT_cmb.value = None
    COU_AGE_MIN_intbox.value = 0
    COU_AGE_MAX_intbox.value = 0
    COU_QUOTA_intbox.value = 0
    COU_STATUS_cmb.value = None
    COU_DATE_datepicker.value = None
    COU_ENT_DATE_datepicker.value = None
    COU_START_TIME_cmb.value = None
    COU_END_TIME_cmb.value = None
    COU_REMARKS_textbox.value = ""
    COU_message_label.value = ""

    COU_add_button.disabled = False
  COU_cancel_button.on_click(on_COU_clear_button_clicked)

  # Back
  def on_COU_back_button_clicked(b):
    showPage("Manager Dashboard")
  COU_back_button.on_click(on_COU_back_button_clicked)

  # Display
  display(widgets.VBox([
      widgets.HBox([COU_title_label]),
      widgets.HBox([COU_Name_textbox]),
      widgets.HBox([COU_DESC_textbox]),
      widgets.HBox([COU_TER_textbox, COU_TER_ID_textbox]),
      widgets.HBox([COU_AGE_MIN_intbox, COU_CAT_cmb]),
      widgets.HBox([COU_AGE_MAX_intbox,COU_QUOTA_intbox]),
      widgets.HBox([COU_DATE_datepicker, COU_STATUS_cmb]),
      widgets.HBox([COU_ENT_DATE_datepicker]),
      widgets.HBox([COU_START_TIME_cmb]),
      widgets.HBox([COU_END_TIME_cmb]),
      widgets.HBox([COU_REMARKS_textbox]),
      widgets.HBox([COU_message_label]),
      widgets.HBox([COU_add_button, COU_back_button, COU_cancel_button])
  ]))
################ Add Course Tab - End ################


##########################################################
################ Edit Course Tab - Begin ################
##########################################################
with tb_all.output_to('Edit Course'):

    # Create widgets for course ID retrieval
    title_label = Label("EDIT COURSE INFORMATION")

    course_id_input = Text(placeholder="Enter Course ID (e.g., C001)", description="Course ID:", layout={'width': '400px'})      # Input field for course ID
    retrieve_btn = widgets.Button(description="Retrieve Course", button_style='primary', layout={'width': '200px'})             # Button to trigger course data retrieval

    spacer = Label("-----------------------------------------------------------------------------------------------------------------------------------")    # Visual separator

    current_header = Label("CURRENT INFORMATION", style={'font_weight': 'bold'}, layout={'width': '350px'})
    edit_header = Label("MAKE CHANGES", style={'font_weight': 'bold'}, layout={'width': '350px'})

    course_name_display = Text(description="Course Name:", disabled=True, layout={'width': '350px'})
    course_name_edit = Text(placeholder="Enter new course name", description="Course Name:", layout={'width': '350px'})

    description_display = widgets.Textarea(description="Description:", disabled=True, layout={'width': '350px', 'height': '80px'})      # Textarea for displaying current course descript
    description_edit = widgets.Textarea(placeholder="Enter new description", description="Description:", layout={'width': '350px', 'height': '80px'})   # Editable textarea for updating course descript

    teacher_display = widgets.Dropdown(description="Teacher:", disabled=True, layout={'width': '350px'})  # Dropdown for displaying current assigned teacher
    teacher_edit = widgets.Dropdown(description="Teacher:", layout={'width': '350px'})     # Editable dropdown for selecting a new teacher from available options

    category_display = widgets.Dropdown(
        options=['Art', 'Math', 'Science', 'Music', 'Dance', 'Sports', 'Language', 'Coding', 'Cooking', 'Drama', 'Yoga', 'Robotics'],
        description="Category:",
        disabled=True,
        layout={'width': '350px'}
    )
    category_edit = widgets.Dropdown(
        options=['Art', 'Math', 'Science', 'Music', 'Dance', 'Sports', 'Language', 'Coding', 'Cooking', 'Drama', 'Yoga', 'Robotics'],
        description="Category:",
        layout={'width': '350px'}
    )

    age_display = widgets.HBox([
        IntText(description="Min Age:", disabled=True, layout={'width': '160px'}),  # Horizontal container for displaying current age range
        IntText(description="Max Age:", disabled=True, layout={'width': '160px'})
    ], layout={'width': '350px'})
    age_edit = widgets.HBox([
        IntText(description="Min Age:", min=4, max=12, layout={'width': '160px'}), # Horizontal container for editing age range
        IntText(description="Max Age:", min=4, max=12, layout={'width': '160px'}) #with validation constraints
    ], layout={'width': '350px'})

    quota_display = IntText(description="Quota:", disabled=True, layout={'width': '350px'})
    quota_edit = IntText(description="Quota:", min=1, max=99, layout={'width': '350px'})
    quota_row = widgets.HBox([quota_display, quota_edit])

    status_display = widgets.Dropdown(
        options=['Active', 'Cancelled', 'Postponed', 'Full', 'Upcoming'],
        description="Status:",
        disabled=True,
        layout={'width': '350px'}
    )
    status_edit = widgets.Dropdown(
        options=['Active', 'Cancelled', 'Postponed', 'Full', 'Upcoming'],
        description="Status:",
        layout={'width': '350px'}
    )

    time_options = [""] + [f"{h:02d}:{m:02d}" for h in range(7,21) for m in (0,30)]   #[""] --> is to make the system works when "Clear" button is clicked

    course_start_time_display = widgets.Dropdown(
        description="Start Time:", options=time_options,
        disabled=True, layout={'width': '350px'}
    )
    course_start_time_edit = widgets.Dropdown(
        description="Start Time:", options=time_options,
        layout={'width': '350px'}
    )

    course_end_time_display = widgets.Dropdown(
        description="End Time:", options=time_options,
        disabled=True, layout={'width': '350px'}
    )
    course_end_time_edit = widgets.Dropdown(
        description="End Time:", options=time_options,
        layout={'width': '350px'}
    )

    course_date_display = DatePicker(description="Course Date:", disabled=True, layout={'width': '350px'})
    course_date_edit = DatePicker(description="Course Date:", layout={'width': '350px'})

    deadline_display = DatePicker(description="Enrollment Deadline:", disabled=True, layout={'width': '350px'})
    deadline_edit = DatePicker(description="Enrollment Deadline:", layout={'width': '350px'})

    remarks_display = widgets.Textarea(description="Remarks:", disabled=True, layout={'width': '350px', 'height': '60px'})
    remarks_edit = widgets.Textarea(placeholder="Enter new remarks", description="Remarks:", layout={'width': '350px', 'height': '60px'})

    message_label = Label("")  # Displaying success/error messages to user

    # Buttons
    clear_btn = widgets.Button(description="Clear Form", button_style='warning', layout={'width': '150px'})
    edit_btn = widgets.Button(description="Update Course", button_style='success', layout={'width': '150px'})
    back_btn = widgets.Button(description="Back to Dashboard", layout={'width': '150px'})

    def update_teacher_dropdown():
        try:
            c.execute("SELECT TER_ID, TER_FNAME, TER_LNAME FROM TEACHER")  # Query database to get all available teachers
            teachers = c.fetchall()
            # Add a blank option at the beginning
            options = [""] + [f"{teacher[1]} {teacher[2]} ({teacher[0]})" for teacher in teachers] # "FirstName LastName (T001)"
            teacher_display.options = options
            teacher_edit.options = options
        except Exception as e:
            message_label.value = f"ERROR: Failed to load teachers - {str(e)}"

    # Age validation
    def validate_age_range():
        if age_edit.children[0].value is not None and age_edit.children[1].value is not None and age_edit.children[0].value > age_edit.children[1].value:
            age_edit.children[0].value = age_edit.children[1].value   # Ensure min age doesn't exceed max age

    # Course time validation
    def validate_time_range():
        if (course_start_time_edit.value and course_end_time_edit.value and
            course_start_time_edit.value >= course_end_time_edit.value):
            message_label.value = "ERROR: Start time must be before end time!"
            return False
        return True


    def on_retrieve_click(b):
        course_id = course_id_input.value.strip()
        if not course_id:
            message_label.value = "ERROR: Please enter a Course ID"
            return

        try:
            # NOTE: bind variable prevents SQL-injection
            c.execute("""
                SELECT COU_ID, COU_NAME, COU_DESC, COU_QUOTA, COU_DATE,
                       COU_ENROLLMENT_DEADLINE, COU_STATUS, COU_AGE_MAX,
                       COU_AGE_MIN, COU_REMARKS, COU_CATEGORY,
                       TEACHER_TER_ID, COU_START_TIME, COU_END_TIME
                FROM COURSES WHERE COU_ID = :1
            """, [course_id])
            row = c.fetchone()

            if row:
                # ---------- DISPLAY (left) ----------
                course_name_display.value      = row[1] or ""
                description_display.value      = row[2] or ""
                quota_display.value            = row[3] or 0
                course_date_display.value      = row[4].date() if row[4] else None
                deadline_display.value         = row[5].date() if row[5] else None
                course_start_time_display.value= row[12] or ""
                course_end_time_display.value  = row[13] or ""
                status_display.value           = row[6] or "Active"
                age_display.children[1].value  = row[7] or 8   # max
                age_display.children[0].value  = row[8] or 4   # min
                remarks_display.value          = row[9] or ""
                category_display.value         = row[10] or "Art"

                # ---------- EDIT (right) – mirror the same values ----------
                course_name_edit.value         = row[1] or ""
                description_edit.value         = row[2] or ""
                quota_edit.value               = row[3] or 0
                course_date_edit.value         = row[4].date() if row[4] else None
                deadline_edit.value            = row[5].date() if row[5] else None
                course_start_time_edit.value   = row[12] or ""
                course_end_time_edit.value     = row[13] or ""
                status_edit.value              = row[6] or "Active"
                age_edit.children[1].value     = row[7] or 8
                age_edit.children[0].value     = row[8] or 4
                remarks_edit.value             = row[9] or ""
                category_edit.value            = row[10] or "Art"

                # Teacher dropdown
                update_teacher_dropdown()
                cur_opt = next((o for o in teacher_edit.options if row[11] in o), "")
                teacher_display.value = cur_opt
                teacher_edit.value    = cur_opt

                message_label.value = f"SUCCESS: Course {course_id} retrieved successfully"
            else:
                message_label.value = f"ERROR: No course found with ID: {course_id}"
                on_clear_click(None)
        except Exception as e:
            message_label.value = f"ERROR: Database error - {str(e)}"
    retrieve_btn.on_click(on_retrieve_click)

    def on_clear_click(b):
        course_id_input.value = ""

        # ----- DISPLAY side (disabled widgets) -----
        course_name_display.value      = ""
        description_display.value      = ""
        teacher_display.value          = ""
        category_display.value         = "Art"
        quota_display.value            = 0
        status_display.value           = "Active"
        course_date_display.value      = None
        deadline_display.value         = None
        course_start_time_display.value = ""
        course_end_time_display.value   = ""
        remarks_display.value          = ""

        # ----- EDIT side -----
        course_name_edit.value         = ""
        description_edit.value         = ""
        teacher_edit.value             = ""
        category_edit.value            = "Art"
        quota_edit.value               = 0
        status_edit.value              = "Active"
        course_date_edit.value         = None
        deadline_edit.value            = None
        course_start_time_edit.value   = ""
        course_end_time_edit.value     = ""
        remarks_edit.value             = ""

        # Age defaults
        age_display.children[0].value = 4
        age_display.children[1].value = 8
        age_edit.children[0].value    = 4
        age_edit.children[1].value    = 8

        message_label.value = ""
    clear_btn.on_click(on_clear_click)


    def on_edit_click(b):
        course_id = course_id_input.value.strip()
        if not course_id:
            message_label.value = "ERROR: Please enter a Course ID first"
            return

        # required fields
        req = [course_name_edit.value, description_edit.value, teacher_edit.value,
               age_edit.children[0].value, age_edit.children[1].value,
               quota_edit.value, course_date_edit.value, deadline_edit.value,
               course_start_time_edit.value, course_end_time_edit.value]
        if not all(req):
            message_label.value = "ERROR: Please fill in all required fields"
            return

        # length checks
        if len(course_name_edit.value) > 50:
            message_label.value = "ERROR: Course name must be ≤ 50 characters"
            return
        if len(description_edit.value) > 500:
            message_label.value = "ERROR: Description must be ≤ 500 characters"
            return
        if remarks_edit.value and len(remarks_edit.value) > 300:
            message_label.value = "ERROR: Remarks must be ≤ 300 characters"
            return

        if not validate_time_range():
            return

        try:
            teacher_id = teacher_edit.value.split('(')[-1].replace(')', '')

            course_date_str = course_date_edit.value.strftime("%Y-%m-%d")
            deadline_str   = deadline_edit.value.strftime("%Y-%m-%d")

            sql = """
                UPDATE COURSES SET
                    COU_NAME = :1, COU_DESC = :2, COU_QUOTA = :3,
                    COU_DATE = TO_DATE(:4,'YYYY-MM-DD'),
                    COU_ENROLLMENT_DEADLINE = TO_DATE(:5,'YYYY-MM-DD'),
                    COU_STATUS = :6, COU_AGE_MAX = :7, COU_AGE_MIN = :8,
                    COU_REMARKS = :9, COU_CATEGORY = :10,
                    TEACHER_TER_ID = :11,
                    COU_START_TIME = :12, COU_END_TIME = :13
                WHERE COU_ID = :14
            """
            c.execute(sql, [
                course_name_edit.value,
                description_edit.value,
                quota_edit.value,
                course_date_str,
                deadline_str,
                status_edit.value,
                age_edit.children[1].value,
                age_edit.children[0].value,
                remarks_edit.value or None,
                category_edit.value,
                teacher_id,
                course_start_time_edit.value,
                course_end_time_edit.value,
                course_id
            ])
            c.execute("COMMIT")

            # sync display side
            for src, dst in [(course_name_edit, course_name_display),
                             (description_edit, description_display),
                             (teacher_edit, teacher_display),
                             (category_edit, category_display),
                             (quota_edit, quota_display),
                             (status_edit, status_display),
                             (course_date_edit, course_date_display),
                             (deadline_edit, deadline_display),
                             (course_start_time_edit, course_start_time_display),
                             (course_end_time_edit, course_end_time_display),
                             (remarks_edit, remarks_display)]:
                dst.value = src.value

            age_display.children[0].value = age_edit.children[0].value
            age_display.children[1].value = age_edit.children[1].value

            message_label.value = f"SUCCESS: Course {course_id} updated"
        except Exception as e:
            message_label.value = f"ERROR: Update failed – {str(e)}"
    edit_btn.on_click(on_edit_click)

    def on_back_click(b):
        showPage("Manager Dashboard")
    back_btn.on_click(on_back_click)

    age_edit.children[0].observe(lambda _: validate_age_range(), names='value')
    age_edit.children[1].observe(lambda _: validate_age_range(), names='value')

    update_teacher_dropdown()
    COUEdit_line1HBox = widgets.HBox([course_id_input, retrieve_btn])

    # <<< CURRENT INFORMATION (left) >>>
    COUEdit_line1VBox = widgets.VBox([
        current_header,               # 1
        course_name_display,          # 2
        description_display,          # 3
        teacher_display,              # 4
        category_display,             # 5
        age_display,                  # 6
        quota_display,                # 7
        status_display,               # 8
        course_date_display,          # 9
        deadline_display,             # 10
        course_start_time_display,    # 12  ← COU_START_TIME
        course_end_time_display,      # 13  ← COU_END_TIME
        remarks_display               # 14
    ])

    # <<< MAKE CHANGES (right) >>>
    COUEdit_line2VBox = widgets.VBox([
        edit_header,                  # 1
        course_name_edit,             # 2
        description_edit,             # 3
        teacher_edit,                 # 4
        category_edit,                # 5
        age_edit,                     # 6
        quota_edit,                   # 7
        status_edit,                  # 8
        course_date_edit,             # 9
        deadline_edit,                # 10
        course_start_time_edit,       # 12  ← COU_START_TIME
        course_end_time_edit,         # 13  ← COU_END_TIME
        remarks_edit                  # 14
    ])

    COUEdit_line2HBox = widgets.HBox([clear_btn, edit_btn, back_btn])
    COUEdit_line3HBox = widgets.HBox([COUEdit_line1VBox, COUEdit_line2VBox])
    message = widgets.HBox([message_label])

    display(widgets.VBox([title_label, COUEdit_line1HBox, spacer,
                          COUEdit_line3HBox, COUEdit_line2HBox, message]))
################ Edit Course Tab - End ################


##########################################################
################ Delete Course Tab - Begin ################
##########################################################

with tb_all.output_to('Delete Course'):

# create 6 label and 5 textboxes
  DELC_id_label = Label("Course ID: ")
  DELC_id_textbox = Text(value='', placeholder='Course ID (e.g. C001)', disabled=False)

  DELC_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  DELC_name_label = Label("Course Name: ")
  DELC_name_textbox = Text(placeholder='Course Name', description='Course Name:', disabled=False)

  DELC_desc_label = Label("Course Description: ")
  DELC_desc_textbox = Text(description='Course Description:', disabled=False)

  DELC_quota_label = Label("Quota: ")
  DELC_quota_intbox = IntText(description='Quota:', disabled=False)

  DELC_cdate_label = Label("Course Date:")
  DELC_cdate_datepicker = DatePicker(description='Course Date:', disabled=False)

  DELC_edate_label = Label("Enrollment Deadline:")
  DELC_edate_datepicker = DatePicker(description='Enrollment Deadline:', disabled=False)


  time_options = [""] + [f"{h:02d}:{m:02d}" for h in range(7,21) for m in (0,30)]

  DELC_cst_label = Label("Course Start Time:")
  DELC_cst_dropdown = widgets.Dropdown(description="Course Start Time:", options=time_options, disabled=False)

  DELC_cdt_label = Label("Course End Time:")
  DELC_cdt_dropdown = widgets.Dropdown(description="Course End Time:", options=time_options, disabled=False)


  DELC_status_label = Label("Status:") # Added label for Status
  DELC_status_dropdown = widgets.Dropdown( # Changed to Dropdown
        options=['Active', 'Not Active'], # Assuming these are the status options
        description="Status:",
        disabled=False # Enabled for display
    )

  DELC_agemin_label = Label("Age Min:")
  DELC_agemin_intbox = IntText(description='Age Min:', disabled=False)

  DELC_agemax_label = Label("Age Max:")
  DELC_agemax_intbox = IntText(description='Age Max:', disabled=False)


  DELC_remarks_label = Label("Remarks:")
  DELC_remarks_textbox = Text(description='Remarks:', disabled=False)

  DELC_category_label = Label("Category:")
  DELC_category_textbox = Text(description='Category:', disabled=False)

  DELC_tea_message_label = Label("")


# create 4 buttons
  DELC_retrieve_button = widgets.Button(description="Retrieve")
  DELC_delete_button = widgets.Button(description="Delete")
  DELC_clear_button = widgets.Button(description="Clear")
  DELC_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
  def on_DELC_retrieve_button_clicked(b):
    c.execute("SELECT * FROM COURSES WHERE cou_id = '" + DELC_id_textbox.value + "'")
    for row in c:
      DELC_name_textbox.value = row[1]
      DELC_desc_textbox.value = row[2]
      DELC_quota_intbox.value = row[3]
      DELC_cdate_datepicker.value = row[4]
      DELC_edate_datepicker.value = row[5]
      DELC_status_dropdown.value = row[6]
      DELC_agemax_intbox.value = row[7]
      DELC_agemin_intbox.value = row[8]
      DELC_remarks_textbox.value = row[9] or ""
      DELC_category_textbox.value = row[10]
      DELC_cst_dropdown.value = row[12]
      DELC_cdt_dropdown.value = row[13]


  DELC_retrieve_button.on_click(on_DELC_retrieve_button_clicked)

  def on_DELC_clear_button_clicked(b):
    DELC_id_textbox.value = ""
    DELC_name_textbox.value = ""
    DELC_desc_textbox.value = ""
    DELC_quota_intbox.value = 0
    DELC_cdate_datepicker.value = None
    DELC_edate_datepicker.value = None
    DELC_status_dropdown.value = 'Active' # Reset dropdown to default
    DELC_agemin_intbox.value = 0 # Changed to 0 for IntText
    DELC_agemax_intbox.value = 0 # Changed to 0 for IntText
    DELC_remarks_textbox.value = "" # Clear Remarks
    DELC_category_textbox.value = "" # Clear Category
    DELC_tea_message_label.value = ""
    DELC_cst_dropdown.value = ""
    DELC_cdt_dropdown.value = ""

  DELC_clear_button.on_click(on_DELC_clear_button_clicked)

  def on_DELC_delete_button_clicked(b):
    c.execute(
    """
          DELETE FROM COURSES
          WHERE cou_id = '{}'
    """.format(DELC_id_textbox.value)
    )
    c.execute("commit")
    DELC_tea_message_label.value = "Delete Success!"
  DELC_delete_button.on_click(on_DELC_delete_button_clicked)

  def on_DELC_back_button_clicked(b):
    showPage("Manager Dashboard")
  DELC_back_button.on_click(on_DELC_back_button_clicked)

# display the objects
  DELC_line1HBox = widgets.HBox([DELC_id_textbox, DELC_retrieve_button])
  DELC_line2HBox = widgets.HBox([DELC_spacer])
  DELC_textboxVBox = widgets.VBox([DELC_name_textbox, DELC_desc_textbox, DELC_quota_intbox, DELC_cdate_datepicker,
                                   DELC_edate_datepicker, DELC_cst_dropdown, DELC_cdt_dropdown, DELC_status_dropdown,
                                   DELC_agemax_intbox, DELC_agemin_intbox, DELC_category_textbox, DELC_remarks_textbox])
  DELC_line3HBox = widgets.HBox([DELC_textboxVBox])
  DELC_line4HBox = widgets.HBox([DELC_delete_button, DELC_clear_button])
  DELC_line5HBox = widgets.HBox([DELC_back_button])

  display(widgets.VBox([DELC_line1HBox, DELC_line2HBox, DELC_line3HBox, DELC_tea_message_label, DELC_line4HBox, DELC_line5HBox]))
################ Delete Course Tab - End ################


##########################################################
################ Course List Page Tab - Begin ################
##########################################################
with tb_all.output_to('Course List Page'):

    # === TITLE ===
    view_course_label = widgets.Label("Available Courses", layout=widgets.Layout(height='50px', font_size='20px', font_weight='bold'))
    view_course_spacer = widgets.Label("----------------------------------------------------------------------------")
    current_course_index = widgets.Label("0")  # Hidden tracker

    # === HEADER LABELS - ONLY ONCE, AT THE TOP! ===
    header_num     = widgets.Label("  #  ", layout=widgets.Layout(width='50px',  justify_content='center'))
    header_id      = widgets.Label("Course ID",     layout=widgets.Layout(width='150px'))
    header_name    = widgets.Label("Course Name",   layout=widgets.Layout(width='250px'))
    header_date    = widgets.Label("Date",          layout=widgets.Layout(width='200px'))
    header_quota   = widgets.Label("Quota",         layout=widgets.Layout(width='150px'))
    header_age     = widgets.Label("Age Range",     layout=widgets.Layout(width='150px'))
    header_teacher = widgets.Label("Teacher",       layout=widgets.Layout(width='250px'))
    header_detail  = widgets.Label("Action",        layout=widgets.Layout(width='150px'))

    header_row = widgets.HBox([
        header_num, header_id, header_name, header_date,
        header_quota, header_age, header_teacher, header_detail
    ])

    # === DATA ROWS - NO LABELS, NO BORDERS, CLEAN STYLE! ===
    view_course_num_labels = [widgets.Label("", layout=widgets.Layout(width='50px', justify_content='flex-end')) for _ in range(5)]

    view_course_id_textboxes      = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='150px')) for _ in range(5)]
    view_course_name_textboxes    = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='250px')) for _ in range(5)]
    view_course_date_textboxes    = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]
    view_course_quota_textboxes   = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='150px')) for _ in range(5)]
    view_course_age_textboxes     = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='150px')) for _ in range(5)]
    view_course_teacher_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='250px')) for _ in range(5)]
    view_course_detail_buttons    = [widgets.Button(description="View Detail", disabled=True, layout=widgets.Layout(width='150px')) for _ in range(5)]

    # CLEAN ROWS — NO GRID LINES! JUST LIKE PROFESSOR'S SLIDE 15!
    view_course_data_HBoxes = [
        widgets.HBox([
            view_course_num_labels[i],
            view_course_id_textboxes[i],
            view_course_name_textboxes[i],
            view_course_date_textboxes[i],
            view_course_quota_textboxes[i],
            view_course_age_textboxes[i],
            view_course_teacher_textboxes[i],
            view_course_detail_buttons[i]
        ])
        for i in range(5)
    ]

    # === NAVIGATION ===
    view_course_prev_button = widgets.Button(description="Previous 5", disabled=True)
    view_course_next_button = widgets.Button(description="Next 5", disabled=True)
    view_course_back_button = widgets.Button(description="Back")
    view_course_nav_HBox = widgets.HBox([view_course_prev_button, view_course_next_button])

    # set global variable here
    max_course_index = 0

    def showCourses(from_index=1, to_index=5):
        global max_course_index

        c.execute("SELECT COUNT(*) FROM COURSES")
        max_course_index = c.fetchone()[0]  # From SQL fetchone() --> (27,) ; then [0] --> 27 !

        if max_course_index == 0:
            view_course_spacer.value = "No courses available yet."
            for hbox in view_course_data_HBoxes:
                hbox.layout.display = 'none'    # Hide all rows
            return

        # Clear all rows
        for i in range(5):
            view_course_num_labels[i].value = ""
            for widget in view_course_data_HBoxes[i].children[1:]:
                if hasattr(widget, 'value'):
                    widget.value = ""
                elif hasattr(widget, 'disabled'):
                    widget.disabled = True
            view_course_data_HBoxes[i].layout.display = ''


        # Fetch up to 5 courses
        # || in SQL = '+' in Python --> "John" + " " + "Smith"  →  "John Smith"
        c.execute("""
            SELECT ROWNUM AS rn, c.COU_ID, c.COU_NAME,
                   TO_CHAR(c.COU_DATE, 'DD-Mon-YYYY'), c.COU_QUOTA,
                   c.COU_AGE_MIN, c.COU_AGE_MAX,
                   t.TER_FNAME || ' ' || t.TER_LNAME
            FROM COURSES c
            JOIN TEACHER t ON c.TEACHER_TER_ID = t.TER_ID
            ORDER BY c.COU_ID
        """)

        display_index = from_index
        i = 0
        for row in c:
            if from_index <= row[0] <= to_index:
                view_course_num_labels[i].value = f"{display_index}.".rjust(3)    # rjust --> Right-justify this string in a field of 3 characters

                view_course_id_textboxes[i].value      = str(row[1])
                view_course_name_textboxes[i].value    = row[2]
                view_course_date_textboxes[i].value    = row[3]
                view_course_quota_textboxes[i].value   = str(row[4])
                view_course_age_textboxes[i].value     = f"{row[5]}-{row[6]}"
                view_course_teacher_textboxes[i].value = row[7]
                view_course_detail_buttons[i].disabled = False

                i += 1
                display_index += 1
                if i >= 5:
                    break

        # Hide empty rows
        for j in range(i, 5):
            view_course_data_HBoxes[j].layout.display = 'none'

        # Update UI
        view_course_prev_button.disabled = (from_index <= 1)
        view_course_next_button.disabled = (to_index >= max_course_index)
        end_idx = min(to_index, max_course_index)
        view_course_spacer.value = f"Showing {from_index} to {end_idx} of {max_course_index} courses"
        current_course_index.value = str(from_index)

    # === BUTTON HANDLERS ===
    def on_prev(b):
        idx = max(1, int(current_course_index.value) - 5)
        showCourses(idx, idx + 4)
    def on_next(b):
        idx = int(current_course_index.value) + 5
        showCourses(idx, idx + 4)
    def on_back(b):
        showPage("Main Menu")
    '''
    def on_detail(b):
        for i in range(5):
            if view_course_detail_buttons[i] == b:
                print(f"Viewing details for Course ID: {view_course_id_textboxes[i].value}")
                break
    '''

    # BUTTON handlers for View detail that directs to the view course detail page
    def on_view_course_detail_button_clicked(b):

      # 1) find which button was clicked (Lab style)
      row_index = None
      for i in range(5):
        if view_course_detail_buttons[i] is b:
            row_index = i
            break
      if row_index is None:
            return

      course_id_selected = view_course_id_textboxes[row_index].value.strip() # we assign the value of that index view detail we find  to the course id

      if course_id_selected == "":
        return

      # 2) prepare values in the detail page (fill ID first)
      cd_course_id.value = course_id_selected # this is just to corresponds to my cd_course_id in the view course detail

      # 3) retrieve the full record for that course
      c.execute("""
          SELECT
              c.COU_NAME,
              c.COU_CATEGORY,
              c.COU_DESC,
              c.COU_REMARKS,
              t.TER_FNAME || ' ' || t.TER_LNAME AS TEACHER,
              c.COU_AGE_MIN,
              c.COU_AGE_MAX,
              c.COU_DATE,
              c.COU_ENROLLMENT_DEADLINE
            FROM COURSES c
            JOIN TEACHER t ON c.TEACHER_TER_ID = t.TER_ID
            WHERE c.COU_ID = '{}'
      """.format(course_id_selected)) # Python string formatting



      row = c.fetchone()
      if row:
        # unpack
        (name, category, desc, remark, teacher, min_age, max_age, course_dt, deadline_dt) = row

        # assign to your cd_* widgets
        cd_course_name.value = name or ""
        cd_category.value    = category or ""
        cd_desc.value        = desc or ""
        cd_remark.value      = remark or ""
        cd_teacher.value     = teacher or ""
        cd_min_age.value     = int(min_age) if min_age is not None else 0
        cd_max_age.value     = int(max_age) if max_age is not None else 0

        # DatePicker expects a datetime.date, in line 2138, I set it as a datepicker, so some conversion is needed.
        try:
            cd_course_date.value = course_dt.date() if hasattr(course_dt, "date") else None # Checks if the object (from SQL) has a .date() method. If yes → it’s a full datetime, so we can extract just the date part.
        except:
            cd_course_date.value = None
        try:
            cd_deadline.value = deadline_dt.date() if hasattr(deadline_dt, "date") else None
        except:
            cd_deadline.value = None

      # 4) navigate to the detail page (Lab showPage)

      showPage("Course Detail Page")
      refresh_enroll_button()

    # re-wire the five buttons to this handler
    for btn in view_course_detail_buttons:
      btn.on_click(on_view_course_detail_button_clicked)


    view_course_prev_button.on_click(on_prev)
    view_course_next_button.on_click(on_next)
    view_course_back_button.on_click(on_back)


    def on_btnCoursePage_button_clicked(b):
        showPage("Course List Page")
        showCourses(1, 5)

    btnCoursePage.on_click(on_btnCoursePage_button_clicked)

    # === FINAL DISPLAY - EXACTLY LIKE PROFESSOR'S SLIDE 15! ===
    view_course_VBox = widgets.VBox([
        view_course_label,
        view_course_spacer,
        header_row,                    # HEADER ON TOP
        *view_course_data_HBoxes,      # CLEAN ROWS — NO GRID!
        view_course_nav_HBox,
        view_course_back_button
    ], layout=widgets.Layout(padding='20px'))

    display(view_course_VBox)
    showCourses(1, 5) #Show course 1-5 initially when this "Course List Page" tab is opened

################ Course List Page Tab - End ################






##########################################################
################ Course Detail Page Tab - Begin ################
##########################################################
with tb_all.output_to('Course Detail Page'):

  # This is the title for the course detail Page
  Course_Detail_label = Label("View Course Detail")
  Course_Detail_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")
  Course_Notice_label = Label(
    "ℹ️ Please take a moment to review the course details carefully — including any restrictions or prerequisites — before you proceed to enroll.",
    layout=widgets.Layout(width='900px'),
    style={'font_weight': 'bold', 'text_color': 'red'}
)

  # Left Column (Basic Info)
  cd_course_id   = Text(description="Course ID:", disabled=True)
  cd_course_name = Text(description="Course Name:", disabled=True)
  cd_category    = Text(description="Category:", disabled=True)
  cd_desc        = Text(description="Description:", disabled=True)  # I need to make description and remark space larger like the UI design, can adjust the width and height later.
  cd_remark      = Text(description="Remark:", disabled=True)

  # Right Column (Teacher & Age & Dates)
  cd_teacher     = Text(description="Teacher:", disabled=True)
  cd_min_age     = IntText(description="Min Age:", disabled=True)
  cd_max_age     = IntText(description="Max Age:", disabled=True)
  cd_course_date = DatePicker(description="Course Date:", disabled=True)
  cd_deadline    = DatePicker(description="Enroll DDL:", disabled=True,layout={'description_width': '120px', 'width': '250px', 'height': 'auto'})

  # Define the 3 buttons - Back to Main Menu/ Add Bookmark / Enroll
  cd_back_main_menu = widgets.Button(description="Back to MainMenu")
  cd_add_bmk   = widgets.Button(description="Add Bookmark")
  cd_enroll    = widgets.Button(description="Enroll")

  # NEW: back to course list
  cd_back_to_list   = widgets.Button(description="Back to Course List")

  # UI layouts for the
  left_side = widgets.VBox([
      Course_Detail_label,
      Course_Detail_spacer,
      Course_Notice_label,
      Course_Detail_spacer,
      cd_course_id,
      cd_course_name,
      cd_category,
      cd_desc,
      cd_remark,
      cd_teacher,
      cd_min_age,
      cd_max_age,
      cd_course_date,
      cd_deadline
    ])

  display(widgets.VBox([
      widgets.HBox([left_side]),
      widgets.HBox([cd_back_main_menu,cd_back_to_list, cd_add_bmk, cd_enroll])
  ]))

  # Event handler to direct back to main menu if the BUTTON "Back to Main Menu is clicked"
  def on_btncd_back_main_menue_button_clicked(b):
    showPage("Main Menu")

  cd_back_main_menu.on_click(on_btncd_back_main_menue_button_clicked)

  def on_btncd_back_to_list_clicked(b):
    showPage("Course List Page")
  cd_back_to_list.on_click(on_btncd_back_to_list_clicked)



  def refresh_enroll_button():
    """Enable/disable Enroll based on current user + current course."""
    uid = (current_user_id or "").strip()
    cid = (cd_course_id.value or "").strip()

    # default: disabled until we know it's allowed
    cd_enroll.disabled = True

    if not uid or not cid:
        return  # no user or no course → keep disabled

    # already enrolled?
    c.execute("""
        SELECT 1
          FROM ENROLLMENT
         WHERE MEMBER_MEM_ID = :1
           AND COURSES_COU_ID = :2
        """,
        [uid, cid])

    already = c.fetchone() is not None

    cd_enroll.disabled = already



  ###################### Enroll Button event handler###############################
  # event handler when the Enroll button on the View Course Detail page is clicked

  def on_cd_enroll_button_clicked(b):
    global current_user_id

    uid = (current_user_id or "").strip()
    cid = (cd_course_id.value or "").strip()

    #  already enrolled?
    c.execute("""
        SELECT 1
        FROM ENROLLMENT
        WHERE MEMBER_MEM_ID = '{}' AND COURSES_COU_ID = '{}'
    """.format(uid, cid))
    row = c.fetchone() # if this record is not None, it means that this member has already enrolled in discourse

    if row is not None:
        # reuse the page, but say it's a duplicate
        Enroll_success_label.value = "You have already enrolled in this course."
        cd_enroll.disabled = True # We disable the button
        showPage("Enrollment Success Page")
        return

    # next ERL_ID  (E001, E002, ...)
    c.execute("SELECT ERL_ID FROM ENROLLMENT ORDER BY ERL_ID DESC")
    row = c.fetchone()
    ERL_ID = "E001" if row is None else "E{:03d}".format(int(row[0][1:]) + 1)

    # INSERT
    c.execute("""
        INSERT INTO ENROLLMENT
        VALUES ('{}', '{}', '{}', SYSDATE, '{}', '{}')
    """.format(
        ERL_ID,          # ERL_ID
        cid,             # COURSES_COU_ID
        uid,             # MEMBER_MEM_ID
        "Enrolled",        # ERL_STATUS
        "N/A"            # ERL_FEEDBACK
    ))
    c.execute("commit")

    Enroll_success_label.value = "Enrollment recorded successfully. ERL_ID: {}".format(ERL_ID)
    cd_enroll.disabled = True
    showPage("Enrollment Success Page")

  cd_enroll.on_click(on_cd_enroll_button_clicked)


### Event handler for the button Add bookmrk
  def on_cd_add_bmk_clicked(b):
    global current_user_id

    # --- 1. Auto-generate next bookmark ID (same logic as E001, M001 etc.) ---
    c.execute("SELECT BM_ID FROM BOOKMARK ORDER BY BM_ID DESC")
    row = c.fetchone()

    if row is None:
        next_bm_id = "B001"
    else:
        last_id = row[0]             # e.g. "B014"
        num = int(last_id[1:]) + 1   # -> 15
        next_bm_id = f"B{num:03d}"   # -> "B015"

    # --- 2. Get course ID from the course detail page UI ---
    course_id = cd_course_id.value.strip()

    # --- 3. Insert into BOOKMARK table ---
    c.execute(
        """
        INSERT INTO BOOKMARK
        VALUES (:cou_id, :mem_id, :bm_id, SYSDATE)
        """,
        bm_id=next_bm_id,
        mem_id=current_user_id,
        cou_id=course_id
    )
    conn.commit()

    # --- 4. After successful insert, go to Bookmark Added Page ---
    showPage("Bookmark Added Page")


cd_add_bmk.on_click(on_cd_add_bmk_clicked)




################ Course Detail Page Tab - End ################

##########################################################
################ Enrollment Success Page - Begin ################
##########################################################

with tb_all.output_to('Enrollment Success Page'):
  # This is the title for the Enrollment Success Page
  Enroll_success_label = Label("")     # this is first initialized as empty because the label is dynamic, enrollment success/ you already enrolled


  # create 3 buttons
  es_back_main_menu_btn = widgets.Button(description="Back to MainMenu")
  es_back_btn = widgets.Button(description = "Back to Course Detail")
  es_view_enroll_btn = widgets.Button(description = "View Enrollment")

  es_back_main_menu_btn.on_click(lambda b: showPage("Main Menu"))
  es_view_enroll_btn.on_click(lambda b: showPage("View Enrollment"))

  # define the event handlers for the three buttons when they are being clicked

  # event handler for back to main menu
  def on_btnes_back_main_menu_btn_clicked(b):
    showPage("Main Menu")
  es_back_main_menu_btn.on_click(on_btnes_back_main_menu_btn_clicked)

  # event handler for back buttton directing to view course detail
  def on_btnes_back_btn_clicked(b):
    showPage("Course Detail Page")
    refresh_enroll_button()

  es_back_btn.on_click(on_btnes_back_btn_clicked)


  # event handler for view enrollment, the further event handler functions will wait for aggreating Katrin's code
  def on_btnes_view_enroll_btn_clicked(b):
    showPage("View Enrollment")

  es_view_enroll_btn.on_click(on_btnes_view_enroll_btn_clicked)

  # display the objects
  row1 = widgets.HBox([Enroll_success_label])
  row2 = widgets.HBox([es_back_main_menu_btn, es_back_btn, es_view_enroll_btn])
  display(widgets.VBox([row1, row2]))



################ Course Enrollment Success Tab - End ################





##########################################################
################ Bookmark List Page Tab - Begin ##########
##########################################################

with tb_all.output_to('Bookmark List Page'):
   # === TITLE ===
    bm_title = widgets.Label("My Bookmark List",
                              layout=widgets.Layout(height='50px',
                                                    font_size='20px',
                                                    font_weight='bold'))

    # === SEARCH BAR ===
    bm_search_box = widgets.Text(
        placeholder="Search bookmark by entering course ID(UPPDER CASE!)",
        layout=widgets.Layout(width='600px')
    )
    bm_search_button = widgets.Button(
        description="🔍",
        layout=widgets.Layout(width='50px')
    )
    bm_search_row = widgets.HBox([bm_search_box, bm_search_button])


    # this spacer will later show "1 to 5 --------" etc.
    bm_spacer = widgets.Label("------------------------------------------------------------")

    # Header Labels (Top Row)
    header_bmid    = widgets.Label("Bookmark ID", layout=widgets.Layout(width='90px'))
    header_cid     = widgets.Label("Course ID",   layout=widgets.Layout(width='120px'))
    header_name    = widgets.Label("Course Name", layout=widgets.Layout(width='200px'))
    header_date    = widgets.Label("Course Date", layout=widgets.Layout(width='150px'))
    header_quota   = widgets.Label("Quota",       layout=widgets.Layout(width='80px'))
    header_age     = widgets.Label("Age",         layout=widgets.Layout(width='80px'))
    header_teacher = widgets.Label("Teacher",     layout=widgets.Layout(width='120px'))
    header_action  = widgets.Label("Action",      layout=widgets.Layout(width='150px'))

    bm_header_row = widgets.HBox([
        header_bmid, header_cid, header_name, header_date,
        header_quota, header_age, header_teacher, header_action
    ])
    # Display them first

    # Creates 5 disabled textboxes (empty by default).
    bm_bmid_textboxes        = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='90px'))   for _ in range(5)] # _ means “the loop variable is unused”, Creating 5 textboxes for each column
    bm_course_id_textboxes   = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='120px'))  for _ in range(5)]
    bm_course_name_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px'))  for _ in range(5)]
    bm_course_date_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='150px'))  for _ in range(5)]
    bm_quota_textboxes       = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='80px'))   for _ in range(5)]
    bm_age_textboxes         = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='80px'))   for _ in range(5)]
    bm_teacher_textboxes     = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='120px'))  for _ in range(5)]

    bm_view_buttons   = [widgets.Button(description="View",   disabled=True, layout=widgets.Layout(width='70px')) for _ in range(5)]
    bm_delete_buttons = [widgets.Button(description="Delete", disabled=True, layout=widgets.Layout(width='70px')) for _ in range(5)]

    # BUILD 5 ROWS
    bm_data_HBoxes = [
        widgets.HBox([
            bm_bmid_textboxes[i],
            bm_course_id_textboxes[i],
            bm_course_name_textboxes[i],
            bm_course_date_textboxes[i],
            bm_quota_textboxes[i],
            bm_age_textboxes[i],
            bm_teacher_textboxes[i],
            bm_view_buttons[i],
            bm_delete_buttons[i]
        ])
        for i in range(5)
    ]

    # === NAVIGATION BUTTONS (Previous 5 / Next 5) ===
    bm_prev_button = widgets.Button(
        description="Previous 5",
        disabled=True,
        layout=widgets.Layout(width='120px')
    )
    bm_next_button = widgets.Button(
        description="Next 5",
        disabled=True,
        layout=widgets.Layout(width='120px')
    )
    bm_nav_row = widgets.HBox([bm_prev_button, bm_next_button])

    # Hidden Index Tracker, stores the first record number being displayed.1 → showing rows 1–5 6 → showing rows 6–10. 11 → showing rows 11–15
    bm_current_index = widgets.Label("0")

    # === FOOTER BUTTONS ===
    bm_back_button = widgets.Button(
        description="Back to Main Menu",
        layout=widgets.Layout(width='180px')
    )
    bm_my_enroll_button = widgets.Button(
        description="My Enrollment",
        layout=widgets.Layout(width='150px')
    )
    bm_footer_row = widgets.HBox([bm_back_button, bm_my_enroll_button])

    # === DISPLAY EVERYTHING ===
    display(
        widgets.VBox([
            bm_title,
            bm_search_row,
            bm_spacer,
            bm_header_row,
            bm_data_HBoxes[0],
            bm_data_HBoxes[1],
            bm_data_HBoxes[2],
            bm_data_HBoxes[3],
            bm_data_HBoxes[4],
            widgets.Label("------------------------------------------------------------"),
            bm_nav_row,
            bm_footer_row
        ])
    )




##########################################################


# Before showing new bookmark results, wipe all 5 rows clean so old data does NOT stay in the UI.
def bm_clear_display():
    # turn off Prev/Next for now
    bm_prev_button.disabled = True
    bm_next_button.disabled = True

    # clear each row
    for i in range(5):
        bm_bmid_textboxes[i].value        = ""
        bm_course_id_textboxes[i].value   = ""
        bm_course_name_textboxes[i].value = ""
        bm_course_date_textboxes[i].value = ""
        bm_quota_textboxes[i].value       = ""
        bm_age_textboxes[i].value         = ""
        bm_teacher_textboxes[i].value     = ""
        bm_view_buttons[i].disabled       = True
        bm_delete_buttons[i].disabled     = True

    # clear the spacer text
    bm_spacer.value = "------------------------------------------------------------"



# ===== Global variable for Bookmark paging =====
max_bookmark_index = 0


def showBookmarks(from_index, to_index=5):
    """
    Show up to 5 bookmarks for the current member,
    from global index from_index to to_index, similar to showCourses().
    """
    global max_bookmark_index


    # === 1. Count how many bookmarks this member has ===
    c.execute("""
        SELECT COUNT(*)
        FROM BOOKMARK
        WHERE MEMBER_MEM_ID = :mid
    """, mid=current_user_id)

    max_bookmark_index = c.fetchone()[0]   # e.g. (2,) -> 2 Save the number of bookmarks this member has into the variable max_bookmark_index.”

    # If no bookmarks at all
    if max_bookmark_index == 0:
        bm_spacer.value = "No bookmark available yet."
        for hbox in bm_data_HBoxes:
            hbox.layout.display = 'none'   # hide all rows
        return

    # === 2. Clear all rows (same pattern as showCourses) ===
    for i in range(5):
        for widget in bm_data_HBoxes[i].children:
            # clear textbox content if it has "value"
            if hasattr(widget, 'value'):
                widget.value = ""
            # disable buttons / textboxes if they have "disabled"
            if hasattr(widget, 'disabled'):
                widget.disabled = True
        bm_data_HBoxes[i].layout.display = ''   # make sure row is visible

    # === 3. Fetch up to 5 bookmarks for this member ===
    # === 3. Fetch up to 5 bookmarks for this member ===
    # Reuse the same columns as Course List Page + join with BOOKMARK

    # Reuse the same columns as Course List Page + join with BOOKMARK
    c.execute("""
        SELECT * FROM (
          SELECT
            rownum AS rn,                                  -- 0
            b.BM_ID,                                      -- 1
            c.COU_ID,                                     -- 2
            c.COU_NAME,                                   -- 3
            TO_CHAR(c.COU_DATE, 'DD-Mon-YYYY') AS C_DATE, -- 4
            c.COU_QUOTA,                                  -- 5
            (c.COU_AGE_MIN || ' - ' || c.COU_AGE_MAX) AS AGE_RANGE,  -- 6
            (t.TER_FNAME || ' ' || t.TER_LNAME) AS TEACHER_NAME      -- 7
          FROM BOOKMARK b
             JOIN COURSES c ON b.COURSES_COU_ID = c.COU_ID
             JOIN TEACHER t ON c.TEACHER_TER_ID = t.TER_ID
          WHERE b.MEMBER_MEM_ID = :mid
          ORDER BY c.COU_ID
        )
        WHERE rn BETWEEN :from_idx AND :to_idx
      """, mid=current_user_id, from_idx=from_index, to_idx=to_index)




    # === 4. Fill UI rows ===
    display_index = from_index
    i = 0
    for row in c:
        # row[0] = rn (we don't display it)
        bm_bmid_textboxes[i].value        = str(row[1])   # BM_ID
        bm_course_id_textboxes[i].value   = str(row[2])   # COU_ID
        bm_course_name_textboxes[i].value = str(row[3])   # COU_NAME
        bm_course_date_textboxes[i].value = str(row[4])   # COU_DATE (formatted)
        bm_quota_textboxes[i].value       = str(row[5])   # COU_QUOTA
        bm_age_textboxes[i].value         = str(row[6])   # "min - max"
        bm_teacher_textboxes[i].value     = str(row[7])   # "Fname Lname"

        bm_view_buttons[i].disabled   = False
        bm_delete_buttons[i].disabled = False

        i += 1
        display_index += 1
        if i >= 5:
            break
    # === 5. Hide any unused rows (e.g., last page with <5 bookmarks) ===
    for j in range(i, 5):
        bm_data_HBoxes[j].layout.display = 'none'

    # === 6. Update navigation and spacer text ===
    bm_prev_button.disabled = (from_index <= 1)
    bm_next_button.disabled = (to_index >= max_bookmark_index)

    bm_spacer.value = f"Showing {from_index} to {min(to_index, max_bookmark_index)} of {max_bookmark_index} bookmarks"
    bm_current_index.value = str(from_index)




####### If the member entered a course id the the text box.
def showBookmarks_by_course(course_id):
    """
    Show ONLY the bookmarks of the current member for a given course_id.
    Used when the user searches by course ID.
    """
    global max_bookmark_index
    bm_current_index.value = "1"       # start index for this filtered view

    # 1. Count how many bookmarks THIS member has for THIS course
    c.execute("""
        SELECT COUNT(*)
        FROM BOOKMARK
        WHERE MEMBER_MEM_ID  = :mid
          AND COURSES_COU_ID = :cid
    """, mid=current_user_id, cid=course_id)

    max_bookmark_index = c.fetchone()[0]

    # If this member has not bookmarked this course
    if max_bookmark_index == 0:
        bm_clear_display()
        bm_spacer.value = "You have not bookmarked this course yet."
        return

    # 2. Clear all rows and make them visible
    for i in range(5):
        for w in bm_data_HBoxes[i].children:
            if hasattr(w, "value"):
                w.value = ""
            if hasattr(w, "disabled"):
                w.disabled = True
        bm_data_HBoxes[i].layout.display = ''

    # 3. Fetch the bookmark(s) for this course (normally only 1 row)
    c.execute("""
        SELECT * FROM (
            SELECT
                rownum AS rn,                                   -- 0
                b.BM_ID,                                        -- 1
                c.COU_ID,                                       -- 2
                c.COU_NAME,                                     -- 3
                TO_CHAR(c.COU_DATE, 'DD-MON-YYYY') AS COU_DATE, -- 4
                c.COU_QUOTA,                                    -- 5
                (c.COU_AGE_MIN || ' - ' || c.COU_AGE_MAX) AS AGE_RANGE, -- 6
                (t.TER_FNAME || ' ' || t.TER_LNAME) AS TEACHER_NAME     -- 7
            FROM BOOKMARK b
                 JOIN COURSES c ON b.COURSES_COU_ID = c.COU_ID
                 JOIN TEACHER t ON c.TEACHER_TER_ID = t.TER_ID
            WHERE b.MEMBER_MEM_ID  = :mid
              AND c.COU_ID         = :cid
            ORDER BY c.COU_ID
        )
        WHERE rn BETWEEN 1 AND 5
    """, mid=current_user_id, cid=course_id)

    # 4. Fill the first few rows (usually just one row)
    i = 0
    for row in c:
        bm_bmid_textboxes[i].value        = str(row[1])   # BM_ID
        bm_course_id_textboxes[i].value   = str(row[2])   # COU_ID
        bm_course_name_textboxes[i].value = str(row[3])   # COU_NAME
        bm_course_date_textboxes[i].value = str(row[4])   # COU_DATE
        bm_quota_textboxes[i].value       = str(row[5])   # QUOTA
        bm_age_textboxes[i].value         = str(row[6])   # AGE_RANGE
        bm_teacher_textboxes[i].value     = str(row[7])   # TEACHER_NAME

        bm_view_buttons[i].disabled   = False
        bm_delete_buttons[i].disabled = False

        bm_data_HBoxes[i].layout.display = ''
        i += 1
        if i >= 5:
            break

    # Hide any remaining empty rows
    for j in range(i, 5):
        bm_data_HBoxes[j].layout.display = 'none'

    # No paging for filtered view – just disable Prev/Next
    bm_prev_button.disabled = True
    bm_next_button.disabled = True

    bm_spacer.value        = f"Showing {max_bookmark_index} bookmark(s) for course {course_id}"
    bm_current_index.value = "1"


def on_bm_search_button_clicked(b):
    # Get text and normalize to UPPER CASE
    cid = bm_search_box.value.strip().upper()
    bm_search_box.value = cid   # reflect upper-case in UI

    # If empty: show all bookmarks for this member
    if cid == "":
        showBookmarks(1, 5)
        return

    # 1. Check if this course ID exists at all
    c.execute("SELECT COUNT(*) FROM COURSES WHERE COU_ID = :cid", cid=cid)
    exists = c.fetchone()[0]

    if exists == 0:
        # Invalid course ID
        bm_clear_display()
        bm_spacer.value = "The course ID you entered is not valid. Please try again."
        bm_prev_button.disabled = True   # disable paging in this error state
        bm_next_button.disabled = True
        bm_current_index.value = "0"
        return

    # 2. Course exists → show only this member's bookmark(s) for that course
    showBookmarks_by_course(cid)


bm_search_button.on_click(on_bm_search_button_clicked)


def on_bm_prev_button_clicked(b):
    global max_bookmark_index

    # current first record being shown
    current_start = int(bm_current_index.value or "1")   # e.g. "6" → 6

    # move back by 5, but not below 1
    new_from = max(1, current_start - 5)
    new_to   = new_from + 4

    showBookmarks(new_from, new_to)

bm_prev_button.on_click(on_bm_prev_button_clicked)




def on_bm_next_button_clicked(b):
    global max_bookmark_index

    current_start = int(bm_current_index.value or "1")   # e.g. "1" → 1
    new_from = current_start + 5                         # 1 → 6, 6 → 11

    # if we’re already at / after the last page, do nothing
    if new_from > max_bookmark_index:
        return

    new_to = new_from + 4
    showBookmarks(new_from, new_to)

bm_next_button.on_click(on_bm_next_button_clicked)



def bm_delete_clicked(row_index, b):
    """
    Delete button for row_index (0..4).
    Remove that bookmark from DB and refresh the current view.
    """
    bm_id = bm_bmid_textboxes[row_index].value.strip()
    if bm_id == "":
        # empty row
        return

    # 1. Delete from BOOKMARK table
    c.execute("""
        DELETE FROM BOOKMARK
        WHERE BM_ID         = :bid
          AND MEMBER_MEM_ID = :mid
    """, bid=bm_id, mid=current_user_id)

    conn.commit()   # or your lab’s commit method

    # 2. Refresh the bookmark list
    #    If the search box is empty -> show ALL bookmarks (with paging)
    #    If there is a course ID in the search box -> show filtered result
    cid = bm_search_box.value.strip().upper()

    if cid == "":
        # re-show full list starting from first page
        showBookmarks(1, 5)
    else:
        # re-show filtered bookmarks for that course
        showBookmarks_by_course(cid)



###########################################################
################ Bookmark Delete Handler ###################
###########################################################

# This function is called by every "Delete" button in the 5 rows
def on_bm_delete_button_clicked(b):
    # 1. Identify which row's delete button was clicked
    index = bm_delete_buttons.index(b)

    # 2. Retrieve the Bookmark ID from that row
    bm_id = bm_bmid_textboxes[index].value.strip()   # e.g. "B006"

    if bm_id == "":
        bm_spacer.value = "Error: No Bookmark ID found to delete."
        return

    # 3. DELETE the bookmark from the BOOKMARK table  (Lab format)
    c.execute(f"""
        DELETE FROM BOOKMARK
        WHERE BM_ID = '{bm_id}'
    """)

    c.execute("commit")   # commit after DML — Lab requirement

    # 4. Disable the delete button to prevent double-click
    bm_delete_buttons[index].disabled = True

    # 5. Direct user to the confirmation page
    showPage("Bookmark Deleted Page")

for btn in bm_delete_buttons:
    btn.on_click(on_bm_delete_button_clicked)



### ============================
### Bookmark → View Course handler
### ============================
def on_BMK_view_clicked(row_index):
    global selected_course_id

    # 1. Read the course ID in the clicked row
    selected_course_id = bm_course_id_textboxes[row_index].value.strip()

    if selected_course_id == "":
        print("Error: No course ID found.")
        return

    # 2. Query DB (same style as lab)
    c.execute("""
        SELECT
            c.COU_ID,          -- 0
            c.COU_NAME,        -- 1
            c.COU_CATEGORY,    -- 2
            c.COU_DESC,        -- 3  (course description)
            c.COU_REMARKS,     -- 4
            c.COU_AGE_MIN,     -- 5
            c.COU_AGE_MAX,     -- 6
            c.COU_QUOTA,       -- 7
            c.COU_DATE,        -- 8  (DATE type, for DatePicker)
            t.TER_FNAME || ' ' || t.TER_LNAME AS TEACHER_NAME  -- 9
        FROM COURSES c
             JOIN TEACHER t ON c.TEACHER_TER_ID = t.TER_ID
        WHERE c.COU_ID = :cid
    """, cid=selected_course_id)

    row = c.fetchone()

    if row is None:
        print("Course not found.")
        return

    # helper: convert None -> "" for Text widgets
    def as_text(v):
        return "" if v is None else str(v)

    # 3. Fill the Course Detail Page widgets
    cd_course_id.value   = as_text(row[0])
    cd_course_name.value = as_text(row[1])
    cd_category.value    = as_text(row[2])
    cd_desc.value        = as_text(row[3])
    cd_remark.value      = as_text(row[4])

    # IntText widgets must get ints (or 0)
    cd_min_age.value     = 0 if row[5] is None else int(row[5])
    cd_max_age.value     = 0 if row[6] is None else int(row[6])

    # We are not using quota (row[7]) on this page

    # DatePicker: DATE / datetime / None is OK
    cd_course_date.value = row[8]

    # Teacher name
    cd_teacher.value     = as_text(row[9])

    # Optional: refresh enroll button state based on this course + current user
    try:
        refresh_enroll_button()
    except NameError:
        # if the function isn't defined yet, just ignore
        pass

    # 4. Navigate to Course Detail Page
    showPage("Course Detail Page")


# Attach handler to the 5 visible bookmark "View" buttons
for i in range(5):
    bm_view_buttons[i].on_click(lambda b, idx=i: on_BMK_view_clicked(idx))












##########################################################
################ Bookmark List Page Tab - End ############
##########################################################



##########################################################
############ Bookmark Added Page Tab - Begin #############
##########################################################

with tb_all.output_to('Bookmark Added Page'):

    # === TITLE ===
    bm_added_title = widgets.Label(
        "Bookmark Added !",
        layout=widgets.Layout(height='80px', font_size='24px', font_weight='bold')
    )

    # === MESSAGE (optional) ===
    bm_added_msg = widgets.Label(
        "Your course has been added to your bookmark list.",
        layout=widgets.Layout(height='40px')
    )

    # === BUTTONS ===
    bm_added_back_button = widgets.Button(
        description="Back to Course List",
        layout=widgets.Layout(width='180px')
    )

    bm_added_view_bm_button = widgets.Button(
        description="View my bookmark record",
        layout=widgets.Layout(width='220px')
    )

    bm_added_buttons_row = widgets.HBox(
        [bm_added_back_button, bm_added_view_bm_button],
        layout=widgets.Layout(margin='20px 0px')
    )

    # === DISPLAY EVERYTHING ===
    display(
        widgets.VBox([
            bm_added_title,
            bm_added_msg,
            bm_added_buttons_row
        ])
    )

def on_bm_added_back_button_clicked(b):
    # Go back to Course List Page
    showPage("Course List Page")
bm_added_back_button.on_click(on_bm_added_back_button_clicked)


def on_bm_added_view_bm_button_clicked(b):
    # Go directly to Bookmark List Page
    showPage("Bookmark List Page")


bm_added_view_bm_button.on_click(on_bm_added_view_bm_button_clicked)
##########################################################
############ Bookmark Added Page Tab - End ###############
##########################################################





##########################################################
############### Bookmark Deleted Page - Begin ############
##########################################################
with tb_all.output_to('Bookmark Deleted Page'):

    # === TITLE ===
    BMK_DEL_title = widgets.Label(
        "Bookmark Deleted!",
        layout=widgets.Layout(height='50px',
                              font_size='20px',
                              font_weight='bold')
    )

    BMK_DEL_spacer = widgets.Label(
        "----------------------------------------------------------------------------------------------------------------------------------------"
    )

    # === MESSAGE ===
    BMK_DEL_message = widgets.Label(
        "The bookmark has been removed successfully."
    )

    # === 3 BUTTONS ===
    BMK_DEL_back_button = widgets.Button(description="Back")
    BMK_DEL_main_button = widgets.Button(description="Main Menu")
    BMK_DEL_view_enroll_button = widgets.Button(description="View Enrollment")

    # === BUTTON LAYOUT ===
    BMK_DEL_button_HBox = widgets.HBox([
        BMK_DEL_back_button,
        BMK_DEL_main_button,
        BMK_DEL_view_enroll_button
    ])

    # === DISPLAY ORDER ===
    display(BMK_DEL_title)
    display(BMK_DEL_spacer)
    display(BMK_DEL_message)
    display(widgets.Label(""))      # blank line
    display(BMK_DEL_button_HBox)


    ##########################################################
    # Event handlers
    ##########################################################

    # Back → return to Bookmark List Page
    def on_BMK_DEL_back_clicked(b):
        showPage("Bookmark List Page")

    BMK_DEL_back_button.on_click(on_BMK_DEL_back_clicked)

    # Main Menu → go to main menu
    def on_BMK_DEL_main_clicked(b):
        showPage("Main Menu")

    BMK_DEL_main_button.on_click(on_BMK_DEL_main_clicked)

    # View Enrollment → go to enrollment page
    def on_BMK_DEL_view_enroll_clicked(b):
        showPage("My Enrollment")

    BMK_DEL_view_enroll_button.on_click(on_BMK_DEL_view_enroll_clicked)




showPage("Home Page")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

We have setup two accounts:

(1)
- MEM_ID: M001
- MEM_PW: abc
- MEM_FNAME: Issac
- MEM_LNAME: Wong
- MEM_DOB: 03/01/2005
- K_FNAME: Tom
- K_LNAME: Wong
- K_DOB: 05/01/2010
- MEM_SECURITY_Q: What is your favorite fruit?
- MEM_SECURITY_ANS: Apple
- MEM_TEL: 94338096
- MEM_ADD: The Peak
- MEM_EMAIL: tlwongay@connect.ust.hk
- MEM_REG_DATE: 29/10/2025


(2)
- MEM_ID: M002
- MEM_PW: 1235
- MEM_FNAME: Chris
- MEM_LNAME: BB
- MEM_DOB: 01/01/2005
- K_FNAME: Samuel
- K_LNAME: BB
- K_DOB: 01/01/2010
- MEM_SECURITY_Q: What is your favorite fruit?
- MEM_SECURITY_ANS: Orange
- MEM_TEL: 999
- MEM_ADD: HKUST
- MEM_EMAIL: ust.com
- MEM_REG_DATE: 29/10/2025

In [5]:
# under Next 5 from above:

    # --- View Detail ---
  def on_view_course_detail_button_clicked(b):
        for i in range(5):
            if view_course_detail_buttons[i] == b:
                course_id_selected = view_course_id_textboxes[i].value
                break
        else:
            return

        dtl_course_id_textbox.value = course_id_selected
        c.execute("SELECT * FROM COURSES WHERE COU_ID = :1", (course_id_selected,))
        row = c.fetchone()
        if row:
            dtl_course_name_textbox.value = row[1]
            dtl_course_date_textbox.value = str(row[2]) if row[2] else ""
            dtl_quota_textbox.value = str(row[3]) if row[3] else ""
            min_age = row[4] if row[4] else "?"
            max_age = row[5] if row[5] else "?"
            dtl_age_textbox.value = f"{min_age} - {max_age}"

        showPage("Course Detail")



# Under Attach Handler

    for btn in view_course_detail_buttons:
        btn.on_click(on_view_course_detail_button_clicked)
    btnViewCourse.on_click(on_btnViewCourse_clicked)








IndentationError: unindent does not match any outer indentation level (<tokenize>, line 29)

In [ ]:
import streamlit as st
import pandas as pd

# Sample data - replace this with your actual data source (e.g., from a database or CSV)
teachers = pd.DataFrame({
    'name': ['Teacher1', 'Teacher2', 'Teacher3', 'Teacher4', 'Teacher5'],
    'subject': ['Math', 'Science', 'English', 'History', 'Art'],
    'details': ['Details about Teacher1: Age 35, Experience 10 years.',
                'Details about Teacher2: Age 42, Experience 15 years.',
                'Details about Teacher3: Age 28, Experience 5 years.',
                'Details about Teacher4: Age 50, Experience 25 years.',
                'Details about Teacher5: Age 31, Experience 7 years.']
})

st.title('View Teacher Page')

# Retrieve the whole rows of the first five teachers
first_five = teachers.head(5)

# Display the first five teachers in a table-like format
st.subheader('First Five Teachers')
for i, row in first_five.iterrows():
    cols = st.columns([2, 2, 1])  # Columns for name, subject, and optional button
    with cols[0]:
        st.write(row['name'])
    with cols[1]:
        st.write(row['subject'])
    if i == 2:  # 0-based index, so 3rd row is index 2
        with cols[2]:
            if st.button('View', key=f'view_{i}'):
                st.subheader(f'More Details for {row["name"]}')
                st.write(row['details'])  # Display additional details when button is clicked

# Optionally, print the raw rows to console for debugging (not shown in the app)
print("Retrieved rows of first five teachers:")
print(first_five)

In [ ]:

##########################################################
################ Delete Course Tab - Begin ################
##########################################################

with tb_all.output_to('Delete Course'):

# create 6 label and 5 textboxes
  DELC_id_label = Label("Course ID")
  DELC_id_textbox = Text(value='', placeholder='Course ID (e.g. C001)', disabled=False)

  DELC_spacer = Label("---------------------------------------------------------------------------------------------------------------------------------")

  DELC_name_label = Label("Course Name: ")
  DELC_name_textbox = Text(placeholder='Course Name', description='Course Name:', disabled=False)

  DELC_desc_label = Label("Course Description: ")
  DELC_desc_textbox = Text(description='Course Description:', disabled=False)

  DELC_quota_label = Label("Quota: ")
  DELC_quota_intbox = IntText(description='Quota:', disabled=False)

  DELC_cdate_label = Label("Course Date:")
  DELC_cdate_datepicker = DatePicker(description='Course Date:', disabled=False)

  DELC_edate_label = Label("Enrollment Deadline:")
  DELC_edate_datepicker = DatePicker(description='Enrollment Deadline:', disabled=False)

  DELC_status_label = Label("Status:") # Added label for Status
  DELC_status_dropdown = widgets.Dropdown( # Changed to Dropdown
        options=['Active', 'Not Active'], # Assuming these are the status options
        description="Status:",
        disabled=False # Enabled for display
    )

  DELC_agemin_label = Label("Age Min:")
  DELC_agemin_intbox = IntText(description='Age Min:', disabled=False)
  DELC_agemax_label = Label("Age Max:")
  DELC_agemax_intbox = IntText(description='Age Max:', disabled=False)

  DELC_remarks_label = Label("Remarks:")
  DELC_remarks_textbox = Text(description='Remarks:', disabled=False)

  DELC_category_label = Label("Category:")
  DELC_category_textbox = Text(description='Category:', disabled=False)

  DELC_tea_message_label = Label("")


# create 4 buttons
  DELC_retrieve_button = widgets.Button(description="Retrieve")
  DELC_delete_button = widgets.Button(description="Delete")
  DELC_clear_button = widgets.Button(description="Clear")
  DELC_back_button = widgets.Button(description="Back")

# define the event handlers for the three buttons when they are being clicked
  def on_DELC_retrieve_button_clicked(b):
    c.execute("SELECT * FROM COURSES WHERE cou_id = '" + DELC_id_textbox.value + "'")
    for row in c:
      DELC_name_textbox.value = row[1]
      DELC_desc_textbox.value = row[2]
      DELC_quota_intbox.value = row[3]
      DELC_cdate_datepicker.value = row[4]
      DELC_edate_datepicker.value = row[5]
      DELC_status_dropdown.value = row[6] # Assuming status is at index 6 and setting dropdown value
      DELC_agemax_intbox.value = row[7] # Assuming age min is at index 7
      DELC_agemin_intbox.value = row[8] # Assuming age max is at index 8
      DELC_remarks_textbox.value = row[9] # Assuming remarks is at index 9
      DELC_category_textbox.value = row[10] # Assuming category is at index 10


  DELC_retrieve_button.on_click(on_DELC_retrieve_button_clicked)

  def on_DELC_clear_button_clicked(b):
    DELC_id_textbox.value = ""
    DELC_name_textbox.value = ""
    DELC_desc_textbox.value = ""
    DELC_quota_intbox.value = 0
    DELC_cdate_datepicker.value = None
    DELC_edate_datepicker.value = None
    DELC_status_dropdown.value = 'Active' # Reset dropdown to default
    DELC_agemin_intbox.value = 0 # Changed to 0 for IntText
    DELC_agemax_intbox.value = 0 # Changed to 0 for IntText
    DELC_remarks_textbox.value = "" # Clear Remarks
    DELC_category_textbox.value = "" # Clear Category
    DELC_tea_message_label.value = ""

  DELC_clear_button.on_click(on_DELC_clear_button_clicked)

  def on_DELC_delete_button_clicked(b):
    c.execute(
    """
          DELETE FROM COURSES
          WHERE cou_id = '{}'
    """.format(DELC_id_textbox.value)
    )
    c.execute("commit")
    DELC_tea_message_label.value = "Delete Success!"
  DELC_delete_button.on_click(on_DELC_delete_button_clicked)

  def on_DELC_back_button_clicked(b):
    showPage("Manager Dashboard")
  DELC_back_button.on_click(on_DELC_back_button_clicked)

# display the objects
  DELC_line1HBox = widgets.HBox([DELC_id_textbox, DELC_retrieve_button])
  DELC_line2HBox = widgets.HBox([DELC_spacer])
  DELC_textboxVBox = widgets.VBox([DELC_name_textbox, DELC_desc_textbox, DELC_quota_intbox, DELC_cdate_datepicker, DELC_edate_datepicker, DELC_status_dropdown, DELC_agemax_intbox, DELC_agemin_intbox, DELC_remarks_textbox, DELC_category_textbox]) # Swapped order of agemax and agemin
  DELC_line3HBox = widgets.HBox([DELC_textboxVBox])
  DELC_line4HBox = widgets.HBox([DELC_delete_button, DELC_clear_button])
  DELC_line5HBox = widgets.HBox([DELC_back_button])

  display(widgets.VBox([DELC_line1HBox, DELC_line2HBox, DELC_line3HBox, DELC_tea_message_label, DELC_line4HBox, DELC_line5HBox]))
################ Delete Course Tab - End ################

In [ ]:
##########################################################
################ Member Enrollment Page Tab - Begin ################
##########################################################
with tb_all.output_to('Member Enrollment Page'):
    # === TITLE ===
    mem_enrol_label = widgets.Label("My Enrollment", layout=widgets.Layout(height='50px', font_size='20px', font_weight='bold'))
    mem_enrol_spacer = widgets.Label("----------------------------------------------------------------------------")
    current_course_index = widgets.Label("0") # Hidden tracker

    # === HEADER LABELS - ONLY ONCE, AT THE TOP! ===
    header_num = widgets.Label(" # ", layout=widgets.Layout(width='50px', justify_content='center'))
    header_id = widgets.Label("Course ID",layout=widgets.Layout(width='150px'))
    header_name = widgets.Label("Course Name", layout=widgets.Layout(width='200px'))
    header_date = widgets.Label("Course Date", layout=widgets.Layout(width='200px'))
    header_status = widgets.Label("Status", layout=widgets.Layout(width='200px'))
    header_enrol_date = widgets.Label("Enrollment Date", layout=widgets.Layout(width='200px'))
    header_teacher = widgets.Label("Teacher", layout=widgets.Layout(width='200px'))

    # === DATA ROWS - NO LABELS, NO BORDERS, CLEAN STYLE! ===
    mem_enrol_num_labels = [widgets.Label("", layout=widgets.Layout(width='50px', justify_content='flex-end')) for _ in range(5)]
    mem_enrol_id_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='150px')) for _ in range(5)]
    mem_enrol_name_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]
    mem_enrol_date_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]
    mem_enrol_status_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]
    mem_enrol_enrol_date_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]
    mem_enrol_teacher_textboxes = [widgets.Text(value='', disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]
    mem_enrol_detail_buttons = [widgets.Button(description="View Feedback", disabled=True, layout=widgets.Layout(width='200px')) for _ in range(5)]

    mem_enrol_data_HBoxes = [
        widgets.HBox([
            mem_enrol_num_labels[i],
            mem_enrol_id_textboxes[i],
            mem_enrol_name_textboxes[i],
            mem_enrol_date_textboxes[i],
            mem_enrol_status_textboxes[i],
            mem_enrol_enrol_date_textboxes[i],
            mem_enrol_teacher_textboxes[i],
            mem_enrol_detail_buttons[i]
        ])
        for i in range(5)
    ]

    # === NAVIGATION ===
    mem_enrol_prev_button = widgets.Button(description="Previous 5", disabled=True)
    mem_enrol_next_button = widgets.Button(description="Next 5", disabled=True)
    mem_enrol_back_button = widgets.Button(description="Back")

    # set global variable here
    max_course_index = 0

    def on_btnEnrollPage_clicked(b):
        showPage("Member Enrollment Page")
        showEnrollment(1, 5)

    # Make sure btnEnrollPage is defined somewhere
    # btnEnrollPage.on_click(on_btnEnrollPage_clicked)

    def showEnrollment(from_index=1, to_index=5):
        global max_course_index
        if not current_user_id:
            print("User not logged in!")
            return

        c.execute("SELECT COUNT(*) FROM ENROLLMENT WHERE mem_id = :mem_id", mem_id=current_user_id)
        max_course_index = c.fetchone()[0]

        if max_course_index == 0:
            mem_enrol_spacer.value = "No enrollment record."
            for hbox in mem_enrol_data_HBoxes:
                hbox.layout.display = 'none' # Hide all rows
            return

        # Clear all rows first
        for i in range(5):
            mem_enrol_num_labels[i].value = ""
            mem_enrol_id_textboxes[i].value = ""
            mem_enrol_name_textboxes[i].value = ""
            mem_enrol_date_textboxes[i].value = ""
            mem_enrol_status_textboxes[i].value = ""
            mem_enrol_enrol_date_textboxes[i].value = ""
            mem_enrol_teacher_textboxes[i].value = ""
            mem_enrol_detail_buttons[i].disabled = True
            mem_enrol_data_HBoxes[i].layout.display = ''


        # Get total count of enrollments for this member
        c.execute("SELECT ERL_DATE, ERL_STATUS FROM ENROLLMENT where MEM_ID = '" + current_user_id + "'")

        display_index = 0
        i = 0

        for row in c:
          row_num = row[0]  # The row number from ROW_NUMBER() OVER
          if from_index <= row_num <= to_index and i < 5:
                mem_enrol_status_textboxes[i].value = row[4]
                mem_enrol_enrol_date_textboxes[i].value = row[5]
                i += 1
                display_index += 1
                if i >= 5:
                    break


        # Fetch enrollment data for current member
        sql = """
            SELECT
                ROW_NUMBER() OVER (ORDER BY e.ERL_DATE DESC) AS rn,
                c.COU_ID,
                c.COU_NAME,
                TO_CHAR(c.COU_DATE, 'DD-Mon-YYYY') AS course_date,
                e.ERL_STATUS,
                TO_CHAR(e.ERL_DATE, 'DD-Mon-YYYY') AS enrollment_date,
                t.TER_FNAME || ' ' || t.TER_LNAME AS teacher_name
            FROM ENROLLMENT e
            JOIN COURSES c ON e.COU_ID = c.COU_ID
            JOIN TEACHER t ON c.TEACHER_TER_ID = t.TER_ID
            WHERE e.mem_id = :mem_id
            ORDER BY e.ERL_DATE DESC
        """
        c.execute(sql, mem_id=current_user_id)
        rows = c.fetchall()

        # Display the requested range of rows

        for row in rows:
            if from_index <= row_num <= to_index and i < 5:
                mem_enrol_num_labels[i].value = f"{row_num}.".rjust(3)
                mem_enrol_id_textboxes[i].value = str(row[1])
                mem_enrol_name_textboxes[i].value = row[2]
                mem_enrol_date_textboxes[i].value = row[3]
                mem_enrol_teacher_textboxes[i].value = row[6]
                mem_enrol_detail_buttons[i].disabled = False

        # Hide empty rows
        for j in range(i, 5):
            mem_enrol_data_HBoxes[j].layout.display = 'none'

        # Update UI
        mem_enrol_prev_button.disabled = (from_index <= 1)
        mem_enrol_next_button.disabled = (to_index >= max_course_index)
        end_idx = min(to_index, max_course_index)
        mem_enrol_spacer.value = f"Showing {from_index} to {end_idx} of {max_course_index} enrollments"
        current_course_index.value = str(from_index)

    # === BUTTON HANDLERS ===
    def on_prev(b):
        idx = max(1, int(current_course_index.value) - 5)
        showEnrollment(idx, idx + 4)

    def on_next(b):
        idx = int(current_course_index.value) + 5
        showEnrollment(idx, idx + 4)

    def on_back(b):
        showPage("Main Menu")

    def on_detail(b):
        for i in range(5):
            if mem_enrol_detail_buttons[i] == b:
                print(f"Viewing details for Course ID: {mem_enrol_id_textboxes[i].value}")
                break

    mem_enrol_prev_button.on_click(on_prev)
    mem_enrol_next_button.on_click(on_next)
    mem_enrol_back_button.on_click(on_back)

    for btn in mem_enrol_detail_buttons:
        btn.on_click(on_detail)

    # === FINAL DISPLAY - EXACTLY LIKE PROFESSOR'S SLIDE 15! ===
    mem_enrol_nav_HBox = widgets.HBox([mem_enrol_prev_button, mem_enrol_next_button])
    mem_enrol_header_row = widgets.HBox([
        header_num, header_id, header_name, header_date,
        header_enrol_date, header_teacher, header_status
    ])
    mem_enrol_VBox = widgets.VBox([
        mem_enrol_label,
        mem_enrol_spacer,
        mem_enrol_header_row, # HEADER ON TOP
        *mem_enrol_data_HBoxes, # CLEAN ROWS — NO GRID!
        mem_enrol_nav_HBox,
        mem_enrol_back_button
    ], layout=widgets.Layout(padding='20px'))
    display(mem_enrol_VBox)
